In [2]:
import pymysql
import pandas as pd
import numpy as np
from datetime import datetime
import time
import math

def big_data_center() :
    conn = pymysql.connect(host = "220.95.212.6", 
                           user = "red_dev_jiang", 
                           passwd = "redtable211b",
                           cursorclass=pymysql.cursors.DictCursor, 
                           db = "redtable_bigdata_center",
                           charset='utf8')
    return conn

def raw_db_conn():
    conn = pymysql.connect(host='db-7ma06.pub-cdb.ntruss.com', user='redtable', password='fpemxpdlqmf5491!@#',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "redtable_raw_2022")
    return conn


def dev_db_conn(db) :
    conn = pymysql.connect(host='db-7ma06.pub-cdb.ntruss.com', user='redtable_openapi', password='fpemxpdlqmf5491!@#',
                       autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = db)
    return conn

def live_db_conn() :
    conn = pymysql.connect(host='db-6j3k3.pub-cdb.ntruss.com', user='redtable', password='fpemxpdlqmf5491!@#',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "redtable2021")
    return conn

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_17536\589252130.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


* 외국인 관광객 인기 맛집 데이터 (RT_FRNR_TURIST_BEST_RSTRNT_INFO_202302) -> 확인 필요 -> 완료
* KFOOD식당서비스정보데이터 (RT_KFD_RSTRNT_SVC_INFO_2022_09) -> 업데이트 안됨 -> 완료
    - 해당되는 store만 업데이트 하면 됨
* 채식 식당 정보(베지테리언)(RT_VGTR_RSTRNT_INFO_2022_09) -> DB에 업데이트 안됨 -> 완료
    - 쿼리문으로 엎으면 될 것
* 유아 동반 가능 식당 정보(RT_INFN_ACP_POSBL_RSTRNT_INFO_2022_09) -> DB에 업데이트 안됨 -> 완료
    - 쿼리문으로 엎으면 될 것
* 지자체 인증 식당 데이터(RT_LOCGOV_CRTFC_RSTRNT_INFO_2023_02) -> award-> 안심식당, 모범식당, 위생등급제, 알뜰매장 어쩌구 -> 완료
    - 인증정보 어떻게 확인하는지 필요
* 무슬림 친화 식당 데이터(RT_MUSLIM_RSTRNT_INFO_2023_02) -> store_id 기준으로 데이터 업데이트만 -> 완료

In [2]:
# 테스트 매장 아이디 (제외해야 함)
ban_store = '10001, 10002, 10003, 10005, 10068, 10069, 948544, 948548, 948549, 948550, 948551, 948553'

In [3]:
city_nm = '''서울특별시
부산광역시
대구광역시
인천광역시
대전광역시
광주광역시
울산광역시
세종특별자치시
수원시
성남시
의정부시
안양시
부천시
광명시
평택시
동두천시
안산시
고양시
과천시
구리시
남양주시
오산시
시흥시
군포시
의왕시
하남시
용인시
파주시
이천시
안성시
김포시
화성시
광주시
양주시
포천시
여주시
춘천시
원주시
강릉시
동해시
태백시
속초시
삼척시
청주시
충주시
제천시
천안시
공주시
보령시
아산시
서산시
논산시
계룡시
당진시
전주시
군산시
익산시
정읍시
남원시
김제시
목포시
여수시
순천시
나주시
광양시
포항시
경주시
김천시
안동시
구미시
영주시
영천시
상주시
문경시
경산시
창원시
진주시
통영시
사천시
김해시
밀양시
거제시
양산시
제주시
서귀포시'''
city_nm = city_nm.split('\n')


In [4]:
# 시군구 다국어 번역 추출

conn = dev_db_conn('dictionary')
curs = conn.cursor()
sql = """
SELECT SHIDO_NM, EN from shido;
"""
curs.execute(sql)
shido = pd.DataFrame(curs.fetchall())
sql = """
SELECT SHIDO_NM,SHIGUNGU_NM,EN from shigungu;
"""
curs.execute(sql)
shigungu = pd.DataFrame(curs.fetchall())
curs.close()
conn.close()
city_en = []
for nm in city_nm:
    shido_tmp = shido[shido['SHIDO_NM']==nm]
    if len(shido_tmp)!=0:
        city_en.append([nm,shido_tmp.iloc[0]['EN']])
    else:
        shigungu_tmp = shigungu[shigungu['SHIGUNGU_NM'].str.contains(nm)]
        if len(shigungu_tmp)!=0:
            city_en.append([nm,shigungu_tmp.iloc[0]['EN'].split(', ')[-1]])
city_nm_en = pd.DataFrame(city_en,columns=['city_nm','city_en'])
#city_nm_en.to_excel('230828_citynm2en.xlsx',index=False)

In [5]:
nm_en = {}
for v in city_nm_en.values:
    nm_en[v[0]]=v[1]

### RT_FRNR_TURIST_BEST_RSTRNT_INFO

In [27]:
data_key = 'RT_FRNR_TURIST_BEST_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

### 신규 테이블 생성시 각주 풀고 진행

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
'''

RT_FRNR_TURIST_BEST_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_FRNR_TURIST_BEST_RSTRNT_INFO의 최근 테이블은 RT_FRNR_TURIST_BEST_RSTRNT_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\n'

In [30]:
data_key = 'RT_FRNR_TURIST_BEST_RSTRNT_INFO'

today = datetime.now()
date_string = today.strftime("%Y%m")

conn = live_db_conn()
curs = conn.cursor()

sql = """
SELECT DISTINCT(store_id) FROM redtable2021.`order` where store_id not in (10001, 10002, 10003,10004, 10005, 10015, 10068, 10069, 948544, 948548, 948549, 948550, 948551, 948553);
"""
curs.execute(sql)
order = pd.DataFrame(curs.fetchall())
print('테스트 데이터를 제외한 총 {}개의 order데이터를 확보하였습니다.'.format(len(order)))
curs.close()
conn.close()

테스트 데이터를 제외한 총 3709개의 order데이터를 확보하였습니다.


In [31]:
conn = live_db_conn()
curs = conn.cursor()

sql = """
SELECT store_id, count(store_id) as count from live_event where store_id not in (10001, 10002, 10003,10004, 10005, 10015, 10068, 10069, 948544, 948548, 948549, 948550, 948551, 948553) group by store_id;
"""
curs.execute(sql)
live_event = pd.DataFrame(curs.fetchall())
curs.close()
conn.close()
print('테스트 데이터를 제외한 총 {}개의 live_event데이터를 확보하였습니다.'.format(len(live_event)))

order['count']=0
print('RT_FRNR_TURIST_BEST_RSTRNT_INFO store를 구성합니다.')
t_list = [100,50,30,20,10,9,8,7,6,5]
for i in t_list:
    tmp = live_event[live_event['count']>i]
    result = pd.concat([order,tmp],axis=0).drop_duplicates('store_id',keep='last')
    if len(result)>8500:
        print('RT_FRNR_TURIST_BEST_RSTRNT_INFO store 구성이 완료되었습니다.')
        break

print('store 정보를 확보합니다.')

테스트 데이터를 제외한 총 298926개의 live_event데이터를 확보하였습니다.
RT_FRNR_TURIST_BEST_RSTRNT_INFO store를 구성합니다.
RT_FRNR_TURIST_BEST_RSTRNT_INFO store 구성이 완료되었습니다.
store 정보를 확보합니다.


In [38]:
conn = live_db_conn()
curs = conn.cursor()

sql = """
SELECT id, currency, name, branch_name, address, address_road, telephone_plain, latitude, longitude
FROM store 
where id not in (10001, 10002, 10003,10004, 10005, 10015, 10068, 10069, 948544, 948548, 948549, 948550, 948551, 948553) 
and id in ({})
AND telephone_plain LIKE '0%'
AND telephone_plain LIKE '%000000%'
and name is not NULL and address is not NULL and address_road is not NULL
and length(telephone_plain) >= 9 AND LENGTH(telephone_plain) < 11 and latitude != 0;
""".format(','.join(result['store_id'].astype(str).unique()))
curs.execute(sql)
road_addr = pd.DataFrame(curs.fetchall())
curs.close()
conn.close()
print('store 정보를 확보하였습니다.')

store 정보를 확보하였습니다.


In [33]:
# 식당만 추리기 위하여 rstr 데이터 가지고와서 비교후 진행

with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()

        sql = """
        SELECT A.RSTR_ID, A.SRC_ID as store_id 
        FROM (SELECT * FROM redtable01.RSTR_CONN where SRC='live_db') A
        INNER JOIN redtable01.RSTR_INDEX RI ON RI.RSTR_ID = A.RSTR_ID
        WHERE A.SRC_ID IN {} AND RI.IS_USE = 1 AND RI.IS_OPEN = 1""".format(tuple(road_addr['id']))

        curs.execute(sql)
        result =  pd.DataFrame(curs.fetchall())

        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
result

쿼리 완료 
쿼리 작동 시간 : 0.18039369583129883


,RSTR_ID,store_id
0,361459,10012
1,33292,10013
2,630004,10014
3,265634,10016
4,33628,10017
...,...,...
6637,300180636,1144099
6638,300187968,1151335
6639,300209733,1178296
6640,300223038,1184316


In [34]:
road_addr = pd.merge(result,road_addr,left_on='store_id',right_on='id',how='inner')

In [35]:
road_addr['name_concat'] = road_addr['name']+' '+road_addr['branch_name'].fillna('')
road_addr['currency']='KOR'
road_addr['latitude'] = road_addr['latitude'].apply(lambda x : str(x)[:10])  # 10자
road_addr['longitude'] = road_addr['longitude'].apply(lambda x : str(x)[:11])
road_addr['city_nm'] = road_addr['address'].apply(lambda x : x.split(' ')[0])
final_result = road_addr[['id','currency','city_nm','name_concat','address','address_road','telephone_plain','latitude','longitude']]#.values.tolist()
final_result['telephone_plain'].replace('02-0000-0000',None,inplace=True)
final_result['telephone_plain'] = final_result['telephone_plain'].str.replace('-','')
final_result

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_17536\94608882.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_result['telephone_plain'].replace('02-0000-0000',None,inplace=True)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_17536\94608882.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['telephone_plain'].replace('02-0000-

,id,currency,city_nm,name_concat,address,address_road,telephone_plain,latitude,longitude
0,10012,KOR,서울특별시,명동정,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,0237895130,37.5615704,126.9884713
1,10013,KOR,서울특별시,명동 서서갈비,서울특별시 중구 을지로2가 199-61,서울특별시 중구 명동7가길 20-8,027761075,37.5625121,126.9859934
2,10014,KOR,서울특별시,봉추찜닭 명동중앙우체국점,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,0237899381,37.5610635,126.9822770
3,10016,KOR,서울특별시,명동한우방 명동본점,서울특별시 중구 을지로2가 199-56,서울특별시 중구 명동9길 37-5,0222678132,37.5653856,126.9839386
4,10017,KOR,서울특별시,설빙 신촌점,서울특별시 서대문구 창천동 33-43,서울특별시 서대문구 연세로 23,023258492,37.5577622,126.9366902
...,...,...,...,...,...,...,...,...,...
6637,1144099,KOR,서울특별시,와우숯불왕갈비,서울특별시 마포구 서교동 364-26,서울특별시 마포구 와우산로21길 31,0269305722,37.5522262,126.9221353
6638,1151335,KOR,서울특별시,잊힐리야 연남,서울특별시 마포구 연남동 387-14,서울특별시 마포구 동교로 262-8,0264044966,37.5627588,126.9254965
6639,1178296,KOR,강원특별자치도,원주 우화,강원특별자치도 원주시 무실동 1850-5,강원특별자치도 원주시 능라동길 78,0338131020,37.3331515,127.9295513
6640,1184316,KOR,서울특별시,빈브라더스 커피하우스 서울,서울특별시 마포구 상수동 354-12,서울특별시 마포구 토정로9길 2,023322005,37.5453955,126.9244557


In [28]:
# insert 전 테이블명 컬럼명 체크

print(col_nm)
print(col_cnt)
print(table_nm)

RSTRNT_ID,COUNTRY_NM,CTY_NM,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO
%s,%s,%s,%s,%s,%s,%s,%s,%s
RT_FRNR_TURIST_BEST_RSTRNT_INFO_202407


In [36]:
final_result = final_result.values.tolist()

conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()

conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

데이터 생성 및 업데이트를 완료하였습니다.


### RT_KFD_RSTRNT_INFO

In [57]:
data_key = "RT_KFD_RSTRNT_INFO"
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')'''

RT_KFD_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_KFD_RSTRNT_INFO의 최근 테이블은 RT_KFD_RSTRNT_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')'

In [80]:
conn = dev_db_conn('redtable01')
curs = conn.cursor()

sql = """
    SELECT T3.SRC_ID AS 'RSTRNT_ID', 'KOR' AS 'COUNTRY_NM',
        REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM',
        T1.RSTR_NM AS 'RSTRNT_NM',
        GROUP_CONCAT(DISTINCT T5.LABEL_NM) AS 'RSTRNT_CTGRY_FLAG_CD', IFNULL(T6.RTI_IDEX, 0) AS 'RSTRNT_AVRG_SCORE_CO',
        RPAD(LEFT(T7.LCNS_NO,6), 14, '0')  AS 'RSTRNT_OPBIZ_DT', 
        IF(T8.GRADE IS NOT NULL, T8.GRADE, NULL) AS 'RSTRNT_SNITAT_GRAD_CD',
        IF(T1.RSTR_LNNO_ADRES = '', NULL, T1.RSTR_LNNO_ADRES) AS 'RSTRNT_LNM_ADDR',
        IF(T1.RSTR_RDNMADR = '', NULL, T1.RSTR_RDNMADR) AS 'RSTRNT_ROAD_NM_ADDR',
        IF(T1.RSTR_TELNO_PLAIN = '', NULL, T1.RSTR_TELNO_PLAIN) AS 'RSTR_TELNO_PLAIN',
        T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
    LEFT JOIN redtable01.RSTR_LABEL T4 ON T3.RSTR_ID = T4.RSTR_ID
    LEFT JOIN redtable01.RSTR_LABEL_MASTER T5 ON T4.LABEL_MASTER_ID = T5.ID
    LEFT JOIN redtable01.RSTR_QLT T6 ON T3.RSTR_ID = T6.RSTR_ID
    LEFT JOIN redtable_raw_2022.foodsafety_restaurant T7 ON T1.BSNS_LCNC_NO = T7.LCNS_NO
    LEFT JOIN redtable01.RSTR_AWARD T8 ON T3.RSTR_ID = T8.RSTR_ID AND T8.TYPE_NM = '위생등급제 지정업소'
    WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
    AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%？%'
    GROUP BY T3.SRC_ID;
    """
curs.execute(sql)
df = pd.DataFrame(curs.fetchall())
df

,RSTRNT_ID,COUNTRY_NM,CTY_NM,RSTRNT_NM,RSTRNT_CTGRY_FLAG_CD,RSTRNT_AVRG_SCORE_CO,RSTRNT_OPBIZ_DT,RSTRNT_SNITAT_GRAD_CD,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTR_TELNO_PLAIN,RSTRNT_LA,RSTRNT_LO
0,10012,KOR,서울특별시,명동정,한식,4.2138394,20110000000000,None,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,0237895130,37.5615320,126.9884905
1,10013,KOR,서울특별시,명동 서서갈비,한식,4.2831890,19910000000000,None,서울특별시 중구 명동2가 23,서울특별시 중구 명동10길 35-10,02-754-7018,37.5625121,126.9859934
2,10014,KOR,서울특별시,봉추찜닭,기타,4.4236489,20170000000000,None,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,0237899381,37.5610636,126.9822771
3,10016,KOR,서울특별시,명동한우방,한식,3.9222200,20070000000000,None,서울특별시 중구 을지로2가 199-56,서울특별시 중구 명동9길 37-5,0222678132,37.5653856,126.9839387
4,10017,KOR,서울특별시,설빙,한식,0E-7,19910000000000,None,서울특별시 서대문구 창천동 33-43,서울특별시 서대문구 연세로 23,023258492,37.5577622,126.9366902
...,...,...,...,...,...,...,...,...,...,...,...,...,...
882979,1191094,KOR,부산광역시,회사식당,기타,0E-7,20240200000000,None,부산광역시 강서구 지사동 1172,부산광역시 강서구 과학산단로 502,None,35.1501773,128.8168102
882980,1191095,KOR,대구광역시,후레쉬케터링 모토닉대구공장,기타,0E-7,20240200000000,None,대구광역시 달서구 신당동 1095-11,대구광역시 달서구 달서대로 530,0533823311,35.8458460,128.4933282
882981,1628785,KOR,부산광역시,차오란,None,0E-7,None,None,부산광역시 해운대구 중동 1829,부산광역시 해운대구 달맞이길 30,None,35.1613327,129.1680138
882982,1628786,KOR,부산광역시,야키토리 해공,None,0E-7,None,None,부산광역시 수영구 민락동 24-6,부산광역시 수영구 민락본동로19번길 30-5,None,35.1582366,129.1271139


In [81]:
df['RSTR_TELNO_PLAIN'] = df['RSTR_TELNO_PLAIN'].str.replace('-','')
df['RSTRNT_LA']=df['RSTRNT_LA'].replace('0',None)
df['RSTRNT_LO']=df['RSTRNT_LO'].replace('0',None)
df['RSTRNT_SNITAT_GRAD_CD'].fillna('',inplace=True)
df['RSTRNT_CTGRY_FLAG_CD'].fillna('',inplace=True)

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_17536\1132659259.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['RSTRNT_SNITAT_GRAD_CD'].fillna('',inplace=True)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_17536\1132659259.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [84]:
final_result = df.values.tolist()

conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()

conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

데이터 생성 및 업데이트를 완료하였습니다.


### RT_KFD_RSTRNT_SVC_INFO

In [85]:
data_key = 'RT_KFD_RSTRNT_SVC_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)


RT_KFD_RSTRNT_SVC_INFO의 최근 테이블을 확인합니다.
확인완료
RT_KFD_RSTRNT_SVC_INFO의 최근 테이블은 RT_KFD_RSTRNT_SVC_INFO_202407입니다.


In [ ]:

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

In [86]:

##### 새 RT_KFD_RSTRNT_SVC_INFO table 생성
conn = pymysql.connect(host='db-7ma06.pub-cdb.ntruss.com', user='redtable_openapi', password='fpemxpdlqmf5491!@#',
                       autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = 'redtable01')
curs = conn.cursor()
start = time.time()

sql = """
SELECT R.ID 
, "Y" AS '다국어메뉴정보제공여부'
, CASE WHEN RA.TYPE_NM = '안심식당' THEN "Y" ELSE "N" END AS "안심식당여부"
, "N" AS '가격대구분코드'
, PRKG_POS_YN "주차"
, DSBR_CVNTL_YN AS "장애인시설여부"
, "0" AS 리뷰수
, DELV_SRVIC_YN '배달'
, DELV_SRVIC_YN AS '포장가능여부'
, CASE WHEN RA.TYPE_NM = '미슐랭' THEN "Y" ELSE "N" END  AS '미슐랭'
, CASE WHEN RA.TYPE_NM = '씨트립' THEN "Y" ELSE "N" END  AS '미식림'
#, RO.FGGG_MENU_OFR_YN 
, 'Y' AS "외국어메뉴정보여부"
FROM RSTR R
LEFT JOIN RSTR_OPTION RO ON RO.RSTR_ID = R.ID
LEFT JOIN RSTR_AWARD RA ON RA.RSTR_ID = R.ID
ORDER BY ID ASC
"""

curs.execute(sql)
result = pd.DataFrame(curs.fetchall())
print('새 데이터를 확보하였습니다.', "\n작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
print('RSTR_ID를 store_id로 변경하기 위한 데이터를 확보합니다.')

새 데이터를 확보하였습니다. 
작동 시간 : 40.96983742713928
RSTR_ID를 store_id로 변경하기 위한 데이터를 확보합니다.


In [87]:
# RSTR_ID에서 live_db의 store_id로 변환
conn = dev_db_conn('redtable01')
curs = conn.cursor()

sql = """
SELECT A.RSTR_ID, A.SRC_ID as store_id FROM (SELECT * FROM redtable01.RSTR_CONN where SRC='live_db') A
"""
curs.execute(sql)
target_store = pd.DataFrame(curs.fetchall())

curs.close()
conn.close()
print('확보를 완료하였습니다.')

result = pd.merge(result, target_store, how='inner',left_on='ID',right_on='RSTR_ID')
result = result.drop_duplicates("ID").drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None]).dropna().sort_values(by='store_id')
orig_id = result['ID']
del target_store
print('가격 구분코드 생성을 위한 데이터를 확보합니다.')


# 가격 구분 코드
conn = dev_db_conn('redtable01')
curs = conn.cursor()
start = time.time()

sql = """
SELECT RSTR_ID, SUM(MENU_PRICE) / COUNT(MENU_PRICE) '평균가'
FROM MENU
WHERE RSTR_ID IN {}
GROUP BY RSTR_ID
""".format(tuple(orig_id))

curs.execute(sql)
d = pd.DataFrame(curs.fetchall())
print('확보를 완료하였습니다.', "\n작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
d = dict(d.values)

def round_up_to_nearest_10000(x):
    if x is None or x == 0:
        return np.nan
    else:
        return math.ceil(x / 10000) * 10000

result['가격대구분코드'] = result['RSTR_ID'].apply(lambda x : d.get(x))
result['가격대구분코드'] = result['가격대구분코드'].apply(round_up_to_nearest_10000)

del d

확보를 완료하였습니다.
가격 구분코드 생성을 위한 데이터를 확보합니다.
확보를 완료하였습니다. 
작동 시간 : 20.775420427322388


In [90]:
result2 = result.drop_duplicates("store_id").drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None]).sort_values(by='store_id')
result2.dropna(subset='가격대구분코드',inplace=True)

In [91]:
final_result = result2[['store_id','다국어메뉴정보제공여부','안심식당여부','가격대구분코드','주차','장애인시설여부','리뷰수','배달','포장가능여부','미슐랭','미식림','외국어메뉴정보여부']].values.tolist()
final_result

[[10012, 'Y', 'N', 30000.0, 'Y', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10014, 'Y', 'N', 30000.0, 'N', 'N', '0', 'Y', 'Y', 'N', 'N', 'Y'],
 [10016, 'Y', 'N', 20000.0, 'N', 'N', '0', 'N', 'N', 'N', 'Y', 'Y'],
 [10018, 'Y', 'N', 40000.0, 'Y', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10019, 'Y', 'Y', 30000.0, 'N', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10020, 'Y', 'N', 20000.0, 'N', 'N', '0', 'Y', 'Y', 'N', 'Y', 'Y'],
 [10021, 'Y', 'N', 10000.0, 'Y', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10022, 'Y', 'N', 50000.0, 'Y', 'Y', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10024, 'Y', 'N', 170000.0, 'N', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10025, 'Y', 'N', 30000.0, 'N', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10026, 'Y', 'N', 30000.0, 'N', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10027, 'Y', 'N', 10000.0, 'N', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10028, 'Y', 'N', 20000.0, 'N', 'Y', '0', 'Y', 'Y', 'N', 'N', 'Y'],
 [10029, 'Y', 'Y', 70000.0, 'Y', 'N', '0', 'N', 'N', 'N', 'N', 'Y'],
 [10030, 'Y', 'N', 30000.0, 'N', 

In [93]:
table_nm

'RT_KFD_RSTRNT_SVC_INFO_202407'

In [94]:
print('전체 데이터를 새 테이블에 insert합니다.')
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

전체 데이터를 새 테이블에 insert합니다.
insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_KFD_RSTRNT_INFO 기반으로 RT_KFD_RSTRNT_SVC_INFO 진행해야함

In [17]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM RT_KFD_RSTRNT_INFO_202407 """

curs.execute(sql)
RT_KFD_RSTRNT_INFO = pd.DataFrame(curs.fetchall())
RT_KFD_RSTRNT_INFO

,RSTRNT_ID,COUNTRY_NM,CTY_NM,RSTRNT_NM,RSTRNT_CTGRY_FLAG_CD,RSTRNT_AVRG_SCORE_CO,RSTRNT_OPBIZ_DT,RSTRNT_SNITAT_GRAD_CD,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO
0,10012,KOR,Seoul,명동정,한식,4.21384,201100,None,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,02-3789-5130,37.561597,126.9885794
1,10013,KOR,Seoul,명동 서서갈비,한식,4.28319,199100,None,서울특별시 중구 명동2가 23,서울특별시 중구 명동10길 35-10,None,37.5624875,126.9859696
2,10014,KOR,Seoul,봉추찜닭,기타,4.42365,201700,None,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,02-3789-9381,37.5610647,126.9822893
3,10016,KOR,Seoul,명동한우방,한식,3.92222,200700,None,서울특별시 중구 을지로2가 199-56,서울특별시 중구 명동9길 37-5,02-2267-8132,37.5653904,126.983946
4,10017,KOR,Seoul,설빙,한식,0.00000,199100,None,서울특별시 서대문구 창천동 33-43,서울특별시 서대문구 연세로 23,02-325-8492,37.5577622,126.9366902
...,...,...,...,...,...,...,...,...,...,...,...,...,...
861506,1191094,KOR,Busan,회사식당,기타,0.00000,202402,None,부산광역시 강서구 지사동 1172,부산광역시 강서구 과학산단로 502,None,35.1501773,128.8168102
861507,1191095,KOR,Daegu,후레쉬케터링 모토닉대구공장,기타,0.00000,202402,None,대구광역시 달서구 신당동 1095-11,대구광역시 달서구 달서대로 530,053-382-3311,35.845846,128.4933282
861508,1628785,KOR,Busan,차오란,None,0.00000,None,None,부산광역시 해운대구 중동 1829,부산광역시 해운대구 달맞이길 30,0507-1356-1253,35.1599375,129.1698125
861509,1628786,KOR,Busan,야키토리 해공,None,0.00000,None,None,부산광역시 수영구 민락동 24-6,부산광역시 수영구 민락본동로19번길 30-5,0507-1479-8334,35.1582344,129.1271137


In [18]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM RT_KFD_RSTRNT_SVC_INFO_202407 """

curs.execute(sql)
RT_KFD_RSTRNT_SVC_INFO = pd.DataFrame(curs.fetchall())
RT_KFD_RSTRNT_SVC_INFO

,RSTRNT_ID,MULTILINGUAL_MENU_INFO_PROVD_AT,SAFETY_RSTRNT_AT,PCRNGE_FLAG_CD,PARKNG_POSBL_AT,DSPSN_FCLTY_AT,REVIEW_CO,DLVR_POSBL_AT,PACKNG_POSBL_AT,MICHELIN_RSTRNT_CN,MEISHILIN_RSTRNT_CN,FGGG_MENU_INFO_AT
0,10012,Y,N,28280.020000,Y,N,0.00000,N,N,N,N,Y
1,10013,Y,N,N,N,N,0.00000,N,N,N,N,Y
2,10014,Y,N,20464.285714,N,N,0.00000,Y,Y,N,N,Y
3,10015,Y,N,N,N,N,0.00000,N,N,N,N,Y
4,10016,Y,N,19567.375887,N,N,0.00000,N,N,N,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
1139123,1191618,Y,N,N,N,N,0.00000,N,N,N,N,Y
1139124,1191619,Y,N,N,N,N,0.00000,N,N,N,N,Y
1139125,1191620,Y,N,N,N,N,0.00000,N,N,N,N,Y
1139126,1191621,Y,N,N,N,N,0.00000,N,N,N,N,Y


In [38]:
a = RT_KFD_RSTRNT_INFO['RSTRNT_ID'].values.tolist()
b = RT_KFD_RSTRNT_SVC_INFO['RSTRNT_ID'].values.tolist()
diff = list(set(b)- set(a))
len(diff)

301633

In [35]:
# diff 리스트를 문자열로 변환
diff_str = ', '.join(map(str, diff))

conn = big_data_center()
curs = conn.cursor()

# SQL 쿼리 생성
sql = f"""
DELETE FROM RT_KFD_RSTRNT_SVC_INFO_202407
WHERE RSTRNT_ID IN ({diff_str});
"""
curs.execute(sql)
conn.commit()

### RT_VGTR_RSTRNT_INFO

In [1232]:
data_key = 'RT_VGTR_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)


# 신규 생성시에는 각주 풀고 사용

print('테이블을 생성합니다.')
'''sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_VGTR_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_VGTR_RSTRNT_INFO의 최근 테이블은 RT_VGTR_RSTRNT_INFO_202407입니다.
테이블을 생성합니다.


'sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [1269]:


# 채식 식당 정보
conn  = dev_db_conn('redtable01')
curs  = conn.cursor()
start = time.time()

sql ="""
SELECT R.ID, GROUP_CONCAT(RLM.LABEL_NM) as LABEL_NM
, R.RSTR_NM, R.RSTR_RDNMADR , R.RSTR_LNNO_ADRES ,
CASE WHEN R.RSTR_TELNO_PLAIN = '' THEN NULL
ELSE R.RSTR_TELNO_PLAIN END AS 'RSTR_TELNO_PLAIN' ,
(CASE WHEN R.RSRV_MTHD_NM IS NOT NULL THEN 'Y' ELSE 'N' END) AS 'RESVE'
, GROUP_CONCAT(DISTINCT M.MENU_NM) as MENU_NM , R.RSTR_LA , R.RSTR_LO
FROM RSTR R 
LEFT JOIN RSTR_LABEL RL ON RL.RSTR_ID = R.ID
LEFT JOIN RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
LEFT JOIN MENU M ON M.RSTR_ID = R.ID
LEFT JOIN MENU_TAG MT ON M.ID = MT.MENU_ID
LEFT JOIN MENU_TAG_MASTER MTM ON MT.MENU_TAG_MASTER_ID = MTM.ID
WHERE MTM.TAG_NM = '채식'
GROUP BY R.ID, R.RSTR_NM, R.RSTR_RDNMADR , R.RSTR_LNNO_ADRES , R.RSTR_TELNO_PLAIN, R.RSTR_LA , R.RSTR_LO
UNION
SELECT R.ID, GROUP_CONCAT(RLM.LABEL_NM)
, R.RSTR_NM, R.RSTR_RDNMADR , R.RSTR_LNNO_ADRES ,
CASE WHEN R.RSTR_TELNO_PLAIN = '' THEN NULL
ELSE R.RSTR_TELNO_PLAIN END AS 'RSTR_TELNO_PLAIN' , 
  (CASE WHEN R.RSRV_MTHD_NM IS NOT NULL THEN 'Y' ELSE 'N' END) AS 'RESVE'
, GROUP_CONCAT(DISTINCT M.MENU_NM) , R.RSTR_LA , R.RSTR_LO
FROM RSTR R 
LEFT JOIN RSTR_LABEL RL ON RL.RSTR_ID = R.ID
LEFT JOIN RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
LEFT JOIN MENU M ON M.RSTR_ID = R.ID
WHERE M.MENU_NM REGEXP '채식|비건|베지테리'
GROUP BY R.ID, R.RSTR_NM, R.RSTR_RDNMADR , R.RSTR_LNNO_ADRES , R.RSTR_TELNO_PLAIN, R.RSTR_LA , R.RSTR_LO;
"""
curs.execute(sql)
df =  pd.DataFrame(curs.fetchall())
print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
curs.close()
conn.close()

df['LABEL_NM'] = df['LABEL_NM'].apply(lambda x: ','.join(list(set(x.split(',')))))
df['MENU_NM'] = df['MENU_NM'].apply(lambda x: ','.join(list(set(x.split(',')))))

print('RSTR_ID를 store_id로 변경하기 위한 데이터를 확보합니다.')

쿼리 완료 
쿼리 작동 시간 : 4.082499980926514
RSTR_ID를 store_id로 변경하기 위한 데이터를 확보합니다.


In [1270]:
df['RSTR_TELNO_PLAIN'] = df['RSTR_TELNO_PLAIN'].str.replace('-','')

In [1282]:
final_result = result[['ID','LABEL_NM','RSTR_NM','RSTR_RDNMADR','RSTR_LNNO_ADRES','RSTR_TELNO_PLAIN','RESVE','MENU_NM','RSTR_LA','RSTR_LO']].values.tolist()

In [1278]:
with live_db_conn() as conn:
    with conn.cursor() as curs:
        start = time.time()
        sql ="""
        SELECT s2.id
        FROM redtable2021.store_label s
        INNER JOIN redtable2021.store s2 ON s2.id = s.store_id
        WHERE s.store_label_master_id IN (126,127);
        """
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)

with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()
        sql = """
        SELECT R.ID,
               RLM.LABEL_NM,
               R.RSTR_NM,
               R.RSTR_RDNMADR,
               R.RSTR_LNNO_ADRES,
               R.RSTR_TELNO_PLAIN,
               (CASE WHEN R.RSRV_MTHD_NM IS NOT NULL THEN 'Y' ELSE 'N' END) AS 'RESVE',
               GROUP_CONCAT(DISTINCT M.MENU_NM) as MENU_NM,
               R.RSTR_LA,
               R.RSTR_LO
        FROM redtable01.RSTR R
        INNER JOIN redtable01.RSTR_CONN R2 ON R2.RSTR_ID = R.ID
        LEFT JOIN redtable01.RSTR_LABEL RL ON RL.RSTR_ID = R.ID
        LEFT JOIN redtable01.RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
        LEFT JOIN redtable01.MENU M ON M.RSTR_ID = R.ID
        LEFT JOIN redtable01.MENU_TAG MT ON M.ID = MT.MENU_ID
        LEFT JOIN redtable01.MENU_TAG_MASTER MTM ON MT.MENU_TAG_MASTER_ID = MTM.ID
        WHERE R2.SRC_ID IN %s AND R2.SRC = 'live_db'
        GROUP BY R.ID, R.RSTR_NM, R.RSTR_RDNMADR, R.RSTR_LNNO_ADRES, R.RSTR_TELNO_PLAIN, R.RSTR_LA, R.RSTR_LO
        UNION
        SELECT R.ID,
               RLM.LABEL_NM,
               R.RSTR_NM,
               R.RSTR_RDNMADR,
               R.RSTR_LNNO_ADRES,
               R.RSTR_TELNO_PLAIN,
               (CASE WHEN R.RSRV_MTHD_NM IS NOT NULL THEN 'Y' ELSE 'N' END) AS 'RESVE',
               GROUP_CONCAT(DISTINCT M.MENU_NM) as MENU_NM,
               R.RSTR_LA,
               R.RSTR_LO
        FROM redtable01.RSTR R
        LEFT JOIN redtable01.RSTR_LABEL RL ON RL.RSTR_ID = R.ID
        LEFT JOIN redtable01.RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
        LEFT JOIN redtable01.MENU M ON M.RSTR_ID = R.ID
        WHERE EXISTS (SELECT 1 FROM redtable01.MENU WHERE R.ID = MENU.RSTR_ID AND MENU.MENU_NM REGEXP '채식|비건|베지테리')
        GROUP BY R.ID, R.RSTR_NM, R.RSTR_RDNMADR, R.RSTR_LNNO_ADRES, R.RSTR_TELNO_PLAIN, R.RSTR_LA, R.RSTR_LO
        """

        curs.execute(sql, (tuple(df['id']),))
        result =  pd.DataFrame(curs.fetchall())
        result['RSTR_TELNO_PLAIN'] = result['RSTR_TELNO_PLAIN'].str.replace('-','')
        dl = result.to_dict(orient='records')
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

쿼리 완료 
쿼리 작동 시간 : 0.03142213821411133
쿼리 완료 
쿼리 작동 시간 : 11.480820894241333


In [1262]:
list_data = []
for item in dl:
    list_data.append([
        item['ID'],
        item['LABEL_NM'],
        item['RSTR_NM'],
        item['RSTR_RDNMADR'],
        item['RSTR_LNNO_ADRES'],
        item['RSTR_TELNO_PLAIN'],
        item['RESVE'],
        item['MENU_NM'],
        item['RSTR_LA'],
        item['RSTR_LO']
    ])

In [1283]:
final_result2 = final_result + list_data
final_result2

[[15582,
  '한식',
  '소반식당',
  '부산광역시 서구 대영로85번길 21',
  '부산광역시 서구 동대신동2가 293-10',
  '0512441046',
  'N',
  '1인 추가시,도토리탕수육(大),도토리탕수육(小),된장찌개,연잎밥,정식,콩나물찜(정식포함/大/3인),콩나물찜(정식포함/小/2인),콩치킨(大),콩치킨(小),표고강정(大),표고강정(小)',
  Decimal('35.1119495'),
  Decimal('129.0199485')],
 [20231,
  '한식',
  '다전',
  '부산광역시 부산진구 신천대로62번길 61',
  '부산광역시 부산진구 부전동 240-12',
  '0518086363',
  'N',
  '녹차 우전',
  Decimal('35.1535559'),
  Decimal('129.0582798')],
 [36852,
  '한식',
  '감로헌',
  '전북특별자치도 전주시 덕진구 권삼득로 247',
  '전북특별자치도 전주시 덕진구 금암동 728-215',
  '0632758811',
  'N',
  '감로밥상 A코스,감로밥상 B코스,감로밥상 C코스,맥주,소주,약선수육(작은),약선수육(중간),약선표고탕수,약선황칠오리전골(예약),인생술~술~찹쌀생주',
  Decimal('35.8394243'),
  Decimal('127.1328249')],
 [50608,
  '한식',
  '향적원사찰음식점',
  '경상북도 경주시 불국로 131',
  '경상북도 경주시 마동 953-21',
  '0547750014',
  'N',
  None,
  Decimal('35.7838914'),
  Decimal('129.3075969')],
 [74935,
  '한식',
  '베지그린',
  '서울특별시 강남구 개포로20길 24-10',
  '서울특별시 강남구 개포동 1234-8',
  '025776316',
  'Y',
  '1인 기준',
  Decimal('37.4769816'),
  Decimal('127.0473607'

In [1284]:
# 중복 제거를 위한 집합과 결과 리스트 초기화
seen = set()
unique_list = []

for item in final_result2:
    # 첫 번째 열과 두 번째 열을 기준으로 중복 확인
    identifier = (item[0], item[1])
    if identifier not in seen:
        seen.add(identifier)
        unique_list.append(item)


In [1286]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_VGTR_RSTRNT_INFO_2023_08")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_VGTR_RSTRNT_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(col_cnt)
print(table_nm)

RSTRNT_ID,RSTRNT_CTGRY_NM,RSTRNT_NM,RSTRNT_ROAD_NM_ADDR,RSTRNT_LNM_ADDR,RSTRNT_TEL_NO,RESVE_POSBL_AT,SLE_VGTR_MENU_INFO_DC,RSTRNT_LA,RSTRNT_LO
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
RT_VGTR_RSTRNT_INFO_202407


In [1287]:
print('전체 데이터를 새 테이블에 insert합니다.')
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT IGNORE INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,unique_list)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')


print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

전체 데이터를 새 테이블에 insert합니다.
insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_MENU_KO_INFO

In [1249]:
data_key = 'RT_RESTAURANT_MENU_KO_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_RESTAURANT_MENU_KO_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_MENU_KO_INFO의 최근 테이블은 RT_RESTAURANT_MENU_KO_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [ ]:
final = result[['translated_name','store_id','id','price','name','tag_name','path','lang']]


In [ ]:
ko = final[final['lang'] == 'ko'].drop(columns='lang')
final_result = ko.values.tolist()

conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


In [ ]:
# '' 값 -> 널값 변경 쿼리문
# 추후 필요시 사용
'''UPDATE RT_VGTR_RSTRNT_INFO_202407 s
SET s.RSTRNT_TEL_NO = NULLIF(s.RSTRNT_TEL_NO, '')
WHERE s.RSTRNT_TEL_NO = '';'''

### RT_INFN_ACP_POSBL_RSTRNT_INFO

In [45]:
data_key = 'RT_INFN_ACP_POSBL_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])
del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_INFN_ACP_POSBL_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_INFN_ACP_POSBL_RSTRNT_INFO의 최근 테이블은 RT_INFN_ACP_POSBL_RSTRNT_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [52]:
conn  = dev_db_conn('redtable01')
curs  = conn.cursor()
start = time.time()

sql ="""
SELECT DISTINCT(RO.RSTR_ID), R.AREA_NM_1, R.AREA_NM_2 'CTY_NM', RLM.LABEL_NM, R.RSTR_NM , R.RSTR_RDNMADR , R.RSTR_LNNO_ADRES, R.RSTR_TELNO_PLAIN, RO.DCRN_YN, ' ' AS CHL_MENU_SLE_AT,  M.MENU_NM, R.RSTR_LA, R.RSTR_LO
FROM redtable01.RSTR_OPTION RO
INNER JOIN redtable01.MENU M ON M.RSTR_ID = RO.RSTR_ID
LEFT JOIN redtable01.RSTR_INDEX RI ON RI.RSTR_ID = RO.RSTR_ID
LEFT JOIN redtable01.RSTR R ON R.ID = RO.RSTR_ID
LEFT JOIN redtable01.RSTR_LABEL RL ON RL.RSTR_ID = RO.RSTR_ID
LEFT JOIN  redtable01.RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
WHERE M.MENU_NM REGEXP '어린이|미취학|유아|아동|키즈' OR DCRN_YN = 'Y'
AND RI.IS_USE = 1 
AND RI.IS_OPEN = 1
AND LENGTH(R.RSTR_TELNO_PLAIN) < 11
AND (R.RSTR_TELNO_PLAIN LIKE '0%' or R.RSTR_TELNO_PLAIN LIKE '1%') ;
"""
curs.execute(sql)
df =  pd.DataFrame(curs.fetchall())
print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
curs.close()
conn.close()

쿼리 완료 
쿼리 작동 시간 : 10.32760739326477


In [53]:
d = df[['RSTR_ID','MENU_NM']].groupby("RSTR_ID").agg(lambda x : ' , '.join(x)).reset_index()
d = dict(d.values)
df = df.drop_duplicates("RSTR_ID")
df['MENU_NM'] = df['RSTR_ID'].apply(lambda x : d.get(x))
#db_cols = ','.join(sc['column_name'])
df_result = df.drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])
df_result['CHL_MENU_SLE_AT'] = 'Y'
df_result['RSTR_TELNO_PLAIN'] = df['RSTR_TELNO_PLAIN'].str.replace('-','')
df_result

,RSTR_ID,AREA_NM_1,CTY_NM,LABEL_NM,RSTR_NM,RSTR_RDNMADR,RSTR_LNNO_ADRES,RSTR_TELNO_PLAIN,DCRN_YN,CHL_MENU_SLE_AT,MENU_NM,RSTR_LA,RSTR_LO
0,367213,부산광역시,부산진구,한식,아이온파티,부산광역시 부산진구 서면로 74,부산광역시 부산진구 부전동 257-3,0516089011,N,Y,"유아(4~7세) , 유아",35.1572031,129.0581298
1,141391,서울특별시,중구,기타,순미네 행복게장,서울특별시 중구 을지로43길 38,서울특별시 중구 을지로6가 18-37,0222682059,N,Y,"어린이돈까스 , 어린이(11세까지)무한세트 , 어린이(11세까지)무한3만세트 , 어...",37.5677105,127.0077855
5,570290,서울특별시,강남구,한식,계절밥상,서울특별시 강남구 영동대로 513,서울특별시 강남구 삼성동 159,0200233413,N,Y,"미취학아동(37개월-7세) , 취학아동(8세-13세) , 계절밥상(미취학아동/37개...",37.5116621,127.0594275
9,50349,전라남도,해남군,한식,명승회센터,전라남도 해남군 문내면 명량로 43-18,전라남도 해남군 문내면 학동리 1315,0615347770,N,Y,"어린이돈가스 , 어린이돈가스",34.5829878,126.3104102
11,220049,전라남도,해남군,양식,레이크하우스,전라남도 해남군 삼산면 대흥사길 180,전라남도 해남군 삼산면 구림리 140-16,0615341771,N,Y,어린이정식,34.4946299,126.6167125
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4845,772097,경기도,성남시,한식,연안식당,경기도 성남시 수정구 산성대로 337,경기도 성남시 수정구 신흥동 2463-5,0317323028,N,Y,"어린이 꼬막 비빔밥 , 어린이 날치알 주먹밥",37.4439102,127.1534832
4847,638344,강원특별자치도,춘천시,한식,가마솥,강원특별자치도 춘천시 신동면 풍류길 12,강원특별자치도 춘천시 신동면 증리 932-3,0332432298,N,Y,미취학어린이(5~7세),37.8175721,127.7163381
4853,622725,서울특별시,광진구,중식,해곤선생,서울특별시 광진구 긴고랑로7길 15,서울특별시 광진구 중곡동 232-23,029296884,N,Y,어린이짜장면,37.5641638,127.0793519
4856,780669,부산광역시,부산진구,일식,스시로,부산광역시 부산진구 중앙대로 672,부산광역시 부산진구 부전동 227-2,0515203715,N,Y,스시로 키즈 셋트,35.1530062,129.0596063


In [54]:
df_result.replace('',None,inplace=True)
df_result.dropna(subset='RSTR_NM',inplace=True)
final_result = df_result.values.tolist()

In [56]:
print('전체 데이터를 새 테이블에 insert합니다.')
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

# 전화번호 수기 체크 요망

전체 데이터를 새 테이블에 insert합니다.
insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_MUSLIM_RSTRNT_INFO
- 무슬림 친화 레스토랑은 기존 관광공사 데이터 기반으로 생성된 테이블입니다.
- 기반 데이터는 redtable_cms 의 halal_tmp* 테이블에서 확인할 수 있습니다.
- 현재는 업데이트 되지 않는 것으로 판단되어, 해당 갱신 주기마다 폐업된 매장 또는 아예 바뀌어버린 매장등을 삭제하는 방향으로 업데이트 하여야 합니다.

In [104]:
data_key = 'RT_MUSLIM_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {}""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

table_nm = '{}_{}'.format(data_key,date_string)
'''
print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')'''


RT_MUSLIM_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_MUSLIM_RSTRNT_INFO의 최근 테이블은 RT_MUSLIM_RSTRNT_INFO_202407입니다.


'\nprint(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')'

In [1193]:
s_r_id = [int(i) for i in summary['RSTRNT_ID'].tolist()]

print('기존 데이터를 기반으로 데이터 갱신을 진행합니다.')
print('live db에서 해당 store_id를 가지는 데이터를 가져옵니다.')

conn = live_db_conn()
curs = conn.cursor()

sql = """SELECT id, latitude, longitude,
       telephone_plain, bswn_telno, name,
       roman_name, branch_name, address, address_road, address_etc
       FROM store 
       where and length(telephone_plain) >= 9 AND LENGTH(telephone_plain) < 11 
       id in {}""".format(tuple(s_r_id))

curs.execute(sql)
r_s = pd.DataFrame(curs.fetchall())

match_list=[]
replace_list = []
for i in range(len(summary)):
    rs_id = int(summary.iloc[i]['RSTRNT_ID'])
    store_df = r_s[r_s['id']==rs_id]
    store_nm = store_df.iloc[0]['name']
    rs_nm = summary.iloc[i]['RSTRNT_NM']
    if store_nm in rs_nm:
        match_list.append(rs_id)
    else:
        print(rs_id,'/',store_nm,'/',rs_nm)
        
print('해당 데이터를 수기로 확인하여 ban_list에 추가하세요.')

기존 데이터를 기반으로 데이터 갱신을 진행합니다.
live db에서 해당 store_id를 가지는 데이터를 가져옵니다.
해당 데이터를 수기로 확인하여 ban_list에 추가하세요.


In [1194]:
ban_list = '''130162 / 장어광장 / 해수전복
130624 / 히말라야 / 두르가 본점
15681 / 딱조아92막창 / 춘천원조숯불닭갈비
169607 / 사비나 / 알라딘케밥
17299 / 하누담 / 덕담
21088 / 사이커피(SA_eCOFFEE) / 삼수장어 두산오거리점
25558 / TACO N TIKKA(타코 엔 티카) / 펀자브인도요리전문점 
261647 / 우선생 양꼬치 / 헬로 인디아 알와하
312473 / 구워야제맛 / 기꾸초밥
318780 / 모닝이츠 / 춘천곰탕
35370 / 불란서찐빵무지개점 / 제주랑
36056 / 낭로 / 럭치미
43648 / 도누각 / (주)파티나라하이비스
538446 / 베로나(VERONA) / 사마르칸트
54264 / 화목토식당 / 일도씨닭갈비 미아점
11455 / 싱삼 / 다미헌
116499 / 퇴사당 / 서교오뎅
47659 / 키친오브인디아 / 키친오브 인디아
52312 / 훼밀리아 / 임피리얼팰리스 패밀리아
57663 / 봄베이 브로이 / 인도야시장
97887 / 라소안나푸르나 / 포탈라 동대문점
805886 / 바른 김치찜&김치찌개 / 문희정커피
789379 / 글로리더안채 / 로맨티카
'''
ban_list = ban_list.split('\n')
ban_list = [i.split(' / ')[0] for i in ban_list]

In [1195]:
summary = summary[~summary['RSTRNT_ID'].isin(ban_list)]

replace_list = []
for i in range(len(summary)):
    rs_id = int(summary.iloc[i]['RSTRNT_ID'])
    store_df = r_s[r_s['id']==rs_id]
    nm = store_df.iloc[0]['name']
    branch = store_df.iloc[0]['branch_name']
    if branch:
        nm = nm+' '+branch
    addr = store_df.iloc[0]['address']
    road = store_df.iloc[0]['address_road']
    tel = store_df.iloc[0]['telephone_plain']
    lat = str(store_df.iloc[0]['latitude'])[:10]  # 10자
    lon = str(store_df.iloc[0]['longitude'])[:11] # 11자리
    replace_list.append([rs_id,nm,addr,road,tel,lat,lon])
    
summary['RSTRNT_ID'] = summary['RSTRNT_ID'].astype(int)

tmp_df = pd.DataFrame(replace_list,columns=['id','RSTRNT_NM','RSTRNT_LNM_ADDR','RSTRNT_ROAD_NM_ADDR','RSTRNT_TEL_NO','RSTRNT_LA','RSTRNT_LO'])
result = pd.merge(tmp_df,summary[['RSTRNT_ID','HALAL_CRTFC_FLAG_NM','CRTFC_INSTT_NM','PRYRM_AT','RSTRNT_IMAGE_URL','COUNTRY_NM','CTY_NM']], left_on='id',right_on='RSTRNT_ID')


In [1196]:
result = result[['id', 'HALAL_CRTFC_FLAG_NM', 'CRTFC_INSTT_NM','RSTRNT_NM', 'RSTRNT_LNM_ADDR', 'RSTRNT_ROAD_NM_ADDR', 'RSTRNT_TEL_NO', 'PRYRM_AT', 'RSTRNT_LA',
        'RSTRNT_LO','RSTRNT_IMAGE_URL',
       'COUNTRY_NM']]
result['CTY_NM'] = result['RSTRNT_LNM_ADDR'].apply(lambda x : x.split(' ')[0])

In [1197]:
result.head(2)

,id,HALAL_CRTFC_FLAG_NM,CRTFC_INSTT_NM,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,PRYRM_AT,RSTRNT_LA,RSTRNT_LO,RSTRNT_IMAGE_URL,COUNTRY_NM,CTY_NM
0,10057,무슬림 프렌들리,한국관광공사,함초간장게장,서울특별시 중구 충무로2가 11-1,서울특별시 중구 명동8가길 27,023181624,N,37.5620005,126.9865637,,KOR,서울특별시
1,102757,무슬림 프렌들리,한국관광공사,헬로인디아,서울특별시 마포구 연남동 566-51,서울특별시 마포구 연남로3길 9,023368425,N,37.5612593,126.9219866,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시


In [1188]:
with dev_db_conn('redtable_raw_2022') as conn:
    with conn.cursor() as curs:
        start = time.time()
        sql ="""
        SELECT s.store_id "RSTRNT_ID" , s.store_label_master_id, s.ogn, s.name "RSTRNT_NM",
        r.RSTR_RDNMADR,r.RSTR_LNNO_ADRES, r.RSTR_TELNO_PLAIN, NULL as 'PRY', r.RSTR_LA, r.RSTR_LO, NULL AS URL , '' AS COUNTRY, r.AREA_NM_1
        FROM redtable_raw_2022.halar_source s
        inner join redtable01.RSTR r ON s.RSTR_ID = r.ID; 
        """
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)

#44:할랄 , 45:무슬림친화

crtfc = { 44 : "할랄 공식 인증" , 45:'무슬림 프렌들리'}

df['store_label_master_id'] = df['store_label_master_id'].map(crtfc)
df['COUNTRY'] = 'KOR'
df.rename(columns={'RSTRNT_ID':'id', 'ogn':'CRTFC_INSTT_NM', 'RSTR_RDNMADR':'RSTRNT_ROAD_NM_ADDR', 'RSTR_LNNO_ADRES':'RSTRNT_LNM_ADDR', 'RSTR_TELNO_PLAIN':'RSTRNT_TEL_NO',
                   'PRY':'PRYRM_AT', 'RSTR_LA':'RSTRNT_LA', 'RSTR_LO' :'RSTRNT_LO', 'URL':'RSTRNT_IMAGE_URL', 'COUNTRY' : 'COUNTRY_NM', 'AREA_NM_1':'CTY_NM', 'store_label_master_id':'HALAL_CRTFC_FLAG_NM'}, inplace=True)
df['RSTRNT_TEL_NO'] = df['RSTRNT_TEL_NO'].str.replace('-','')
df

쿼리 완료 
쿼리 작동 시간 : 0.08581972122192383


,id,HALAL_CRTFC_FLAG_NM,CRTFC_INSTT_NM,RSTRNT_NM,RSTRNT_ROAD_NM_ADDR,RSTRNT_LNM_ADDR,RSTRNT_TEL_NO,PRYRM_AT,RSTRNT_LA,RSTRNT_LO,RSTRNT_IMAGE_URL,COUNTRY_NM,CTY_NM
0,840855,할랄 공식 인증,(사)한국할랄수출협회,케르반레스토랑,서울특별시 강남구 영동대로 513,서울특별시 강남구 삼성동 159,025564746,None,37.5118239,127.0591590,None,KOR,서울특별시
1,844614,할랄 공식 인증,(사)한국할랄수출협회,케르반익스프레스,서울특별시 강남구 테헤란로 152,서울특별시 강남구 역삼동 737,025664767,None,37.5000263,127.0365456,None,KOR,서울특별시
2,118840,할랄 공식 인증,(사)한국할랄수출협회,미스터케밥,서울특별시 용산구 이태원로 192,서울특별시 용산구 이태원동 127-2,027921997,None,37.5343669,126.9953719,None,KOR,서울특별시
3,119218,할랄 공식 인증,(사)한국할랄수출협회,케르반레스토랑,서울특별시 용산구 이태원로 192,서울특별시 용산구 이태원동 127-2,027924767,None,37.5343669,126.9953719,None,KOR,서울특별시
4,122384,할랄 공식 인증,(사)한국할랄수출협회,이드,서울특별시 용산구 우사단로10길 15,서울특별시 용산구 이태원동 137-14,07088998210,None,37.5332644,126.9964179,None,KOR,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,35363,무슬림 프렌들리,제주관광공사,청운식당,제주특별자치도 서귀포시 성산읍 일출로 285,제주특별자치도 서귀포시 성산읍 성산리 192-2,0647823912,None,33.4630440,126.9348802,None,KOR,제주특별자치도
137,35366,무슬림 프렌들리,제주관광공사,용두암바당회국수,제주특별자치도 제주시 서해안로 614,제주특별자치도 제주시 용담삼동 1026-3,0647116754,None,33.5169879,126.5027303,None,KOR,제주특별자치도
138,256396,무슬림 프렌들리,제주관광공사,제주대게회타운,제주특별자치도 제주시 서해안로 638,제주특별자치도 제주시 용담삼동 1003-9,0647479289,None,33.5165309,126.5051314,None,KOR,제주특별자치도
139,816339,무슬림 프렌들리,제주관광공사,코코마마우도점,제주특별자치도 제주시 우도해안길 816,제주특별자치도 제주시 우도면 연평리 1290-2,None,None,33.5131314,126.9577602,None,KOR,제주특별자치도


In [1220]:
final_result = pd.concat([result,df],axis=0)
final_result2 = final_result.drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])
final_result2.drop_duplicates(subset=['id','RSTRNT_NM'], inplace=True)

In [1221]:
final_result2.drop_duplicates(subset=['id','RSTRNT_LNM_ADDR'], inplace=True)
final_result2['RSTRNT_TEL_NO'] = final_result2['RSTRNT_TEL_NO'].str.replace('-','')
final_result2['PRYRM_AT'].fillna('N',inplace=True)

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\1990402149.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_result2['PRYRM_AT'].fillna('N',inplace=True)


In [1218]:
# 수정후 재 업데이트시 이용
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_MUSLIM_RSTRNT_INFO_2023_08")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_MUSLIM_RSTRNT_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(col_cnt)
print(table_nm)

RSTRNT_ID,HALAL_CRTFC_FLAG_NM,CRTFC_INSTT_NM,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,PRYRM_AT,RSTRNT_LA,RSTRNT_LO,RSTRNT_IMAGE_URL,COUNTRY_NM,CTY_NM
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
RT_MUSLIM_RSTRNT_INFO_202407


In [1222]:
final_result2

# id = 913722 삭제하기.

,id,HALAL_CRTFC_FLAG_NM,CRTFC_INSTT_NM,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,PRYRM_AT,RSTRNT_LA,RSTRNT_LO,RSTRNT_IMAGE_URL,COUNTRY_NM,CTY_NM
0,10057,무슬림 프렌들리,한국관광공사,함초간장게장,서울특별시 중구 충무로2가 11-1,서울특별시 중구 명동8가길 27,023181624,N,37.5620005,126.9865637,,KOR,서울특별시
1,102757,무슬림 프렌들리,한국관광공사,헬로인디아,서울특별시 마포구 연남동 566-51,서울특별시 마포구 연남로3길 9,023368425,N,37.5612593,126.9219866,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시
2,103459,무슬림 프렌들리,한국관광공사,겐지스 한양대점,서울특별시 성동구 행당동 16-53,서울특별시 성동구 사근동길 6,0222983900,N,37.5601490,127.0420294,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시
3,1072475,무슬림 프렌들리,(사)한국할랄수출협회,키친오브인디아,서울특별시 송파구 방이동 22,서울특별시 송파구 올림픽로 336,None,N,37.5153977,127.1074580,None,KOR,서울특별시
4,107279,무슬림 프렌들리,한국관광공사,옴레스토랑 마포공덕점,서울특별시 마포구 도화동 556,서울특별시 마포구 마포대로 73,027254848,N,37.5423865,126.9489836,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,97546,무슬림 프렌들리,한국관광공사,머노까머나 신촌점,서울특별시 서대문구 창천동 52-25,서울특별시 서대문구 연세로11길 13,023384343,N,37.5588191,126.9360293,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시
277,98092,무슬림 프렌들리,한국관광공사,루나아시아,서울특별시 강남구 삼성동 152,서울특별시 강남구 삼성로104길 12,025723463,N,37.5108553,127.0557382,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시
278,98963,서울관광 인프라조사,한국관광공사,이치류 홍대본점,서울특별시 마포구 서교동 395-124,서울특별시 마포구 잔다리로3안길 44,0214411312,N,37.5506116,126.9191207,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시
279,99129,포크프리(돼지고기 없음),한국관광공사,러빙헛,서울특별시 강남구 개포동 1218-16,서울특별시 강남구 개포로22길 35,025762158,N,37.4769089,127.0494592,http://a.redtable.kr/cms/get_image.php?file=ha...,KOR,서울특별시


In [1224]:
final_result3 = final_result2.values.tolist()

print('전체 데이터를 새 테이블에 insert합니다.')
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result3)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

전체 데이터를 새 테이블에 insert합니다.
insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_LOCGOV_CRTFC_RSTRNT_INFO
- 해당 데이터도 무슬림 친화식당과 동일하게 폐업된 매장 또는 아예 바뀌어버린 매장을 삭제하는 방향으로 업데이트 합니다.

In [98]:
data_key = 'RT_LOCGOV_CRTFC_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {}""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

table_nm = '{}_{}'.format(data_key,date_string)

RT_LOCGOV_CRTFC_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_LOCGOV_CRTFC_RSTRNT_INFO의 최근 테이블은 RT_LOCGOV_CRTFC_RSTRNT_INFO_202407입니다.


In [28]:
print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')


테이블을 생성합니다.
테이블 생성을 완료하였습니다.
새 테이블에 insert 할 데이터를 확보합니다.


In [100]:
conn = live_db_conn()
curs = conn.cursor()
sql="""SELECT store_id,year,type FROM store_award where `type`='안심식당' or `type`='위생등급제 지정업소' or `type`='어린이 우수판매업소' or `type`='모범음식점';"""
curs.execute(sql)
award = pd.DataFrame(curs.fetchall())

award = award.sort_values(by='year').dropna(subset='year').reset_index(drop=True)
award = award.drop_duplicates(subset='store_id',keep='last')

award_id = award['store_id']

conn = live_db_conn()
curs = conn.cursor()
sql="""SELECT id,latitude,longitude,telephone_plain,name,branch_name,address,address_road,currency 
FROM store 
where (telephone_plain LIKE '0%' or telephone_plain LIKE '1%')
and id in {};""".format(tuple(award_id))
curs.execute(sql)
store = pd.DataFrame(curs.fetchall())

store.dropna(subset=['address_road'],inplace=True)
store

,id,latitude,longitude,telephone_plain,name,branch_name,address,address_road,currency
0,10012,37.56157041880520,126.98847133150600,0237895130,명동정,None,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,KRW
1,10014,37.56106356160560,126.98227707999800,0237899381,봉추찜닭,명동중앙우체국점,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,KRW
2,10018,37.55652399209290,126.93499568622800,023379019,삼호복집,None,서울특별시 서대문구 창천동 62-11,서울특별시 서대문구 연세로5다길 10,KRW
3,10019,37.55811430000000,126.93666940000000,023256633,대포찜닭,신촌점,서울특별시 서대문구 창천동 33-27,서울특별시 서대문구 연세로 27-1,KRW
4,10024,37.55112257149390,126.98786783799300,0234559297,엔그릴,N서울타워점,서울특별시 용산구 용산동2가 1-3,서울특별시 용산구 남산공원길 105,KRW
...,...,...,...,...,...,...,...,...,...
48488,1186128,35.22160730000000,126.84957680000000,0629723377,용우동,첨단점,광주광역시 광산구 쌍암동 687-2,광주광역시 광산구 첨단중앙로182번길 76,KRW
48489,1198890,37.59746630000000,127.09416030000000,024920707,씨유,망우씨너스빌점,서울특별시 중랑구 망우동 495,서울특별시 중랑구 용마산로115길 109,KRW
48490,1201448,36.72013470000000,127.42139360000000,0432177900,씨유,오창양청전원점,충청북도 청주시 청원구 오창읍 양청리 736-1,충청북도 청주시 청원구 오창읍 양청택지1길 49-2,KRW
48491,1202664,35.82191850000000,128.75050710000000,0538110022,이마트24,경산계양타운점,경상북도 경산시 계양동 660-13,경상북도 경산시 원효로26길 8,KRW


In [101]:


conn = dev_db_conn('redtable01')
curs = conn.cursor()
sql="""SELECT CON.store_id,AREA_NM_1,AREA_NM_2,AREA_NM_3 FROM RSTR R
inner join (SELECT A.RSTR_ID, A.SRC_ID as store_id FROM (SELECT * FROM redtable01.RSTR_CONN where SRC='live_db') A) CON on CON.RSTR_ID=R.ID
where store_id in {};""".format(tuple(award_id))
curs.execute(sql)
rst = pd.DataFrame(curs.fetchall())

crtf_area = []
for v in rst.values:
    if v[1]=='서울특별시':
        crtf_area.append([v[0],v[2]])
    else:
        crtf_area.append([v[0],v[1]])
        
crtf_df = pd.DataFrame(crtf_area, columns=['store_id','area'])
store['branch_name'].fillna('',inplace=True)
store['full_name'] = store['name']+' '+store['branch_name']
result = pd.merge(store,crtf_df,left_on='id',right_on='store_id')
result = pd.merge(result,award,left_on='id',right_on='store_id')
result['city_nm'] = result['address'].apply(lambda x : x.split(' ')[0] if x is not None else '')
result['locfov_nm'] = result['address'].apply(lambda x : x.split(' ')[1] if x is not None and len(x)>=8 else '')
result['latitude'] = result['latitude'].apply(lambda x : str(x)[:10])  # 10자
result['longitude'] =result['longitude'].apply(lambda x : str(x)[:11])

result = result.drop_duplicates("id").drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None]).dropna().sort_values(by='id')
result['telephone_plain'] = result['telephone_plain'].str.replace('-','')
final_result = result[['type','id','currency','city_nm','locfov_nm','full_name','address','address_road','telephone_plain','latitude','longitude']].values.tolist()
print('전체 데이터를 새 테이블에 insert합니다.')

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_17536\2200537639.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  store['branch_name'].fillna('',inplace=True)


전체 데이터를 새 테이블에 insert합니다.


In [102]:
table_nm

'RT_LOCGOV_CRTFC_RSTRNT_INFO_202407'

In [103]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO
위 테이블 기반으로 메뉴 진행

In [1326]:
data_key = 'RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {}""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

table_nm = '{}_{}'.format(data_key,date_string)
'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''


RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO의 최근 테이블을 확인합니다.
확인완료
RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO의 최근 테이블은 RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [1327]:
conn = big_data_center()
curs = conn.cursor()
sql="""SELECT * FROM RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_2022_12;"""
curs.execute(sql)
check = pd.DataFrame(curs.fetchall())

In [1328]:
# 해당 RSTRNT_ID 는 store_id 임
conn = big_data_center()
curs = conn.cursor()
sql="""SELECT * FROM RT_LOCGOV_CRTFC_RSTRNT_INFO_202407;"""
curs.execute(sql)
df = pd.DataFrame(curs.fetchall())
df


,LOCGOV_CRTFC_FLAG_NM,RSTRNT_ID,COUNTRY_NM,CTY_NM,LOCGOV_NM,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO
0,모범음식점,10012,KRW,서울특별시,중구,명동정,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,0237895130,37.5615704,126.9884713
1,모범음식점,10014,KRW,서울특별시,중구,봉추찜닭 명동중앙우체국점,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,0237899381,37.5610635,126.9822770
2,모범음식점,10018,KRW,서울특별시,서대문구,삼호복집,서울특별시 서대문구 창천동 62-11,서울특별시 서대문구 연세로5다길 10,023379019,37.5565239,126.9349956
3,안심식당,10019,KRW,서울특별시,서대문구,대포찜닭 신촌점,서울특별시 서대문구 창천동 33-27,서울특별시 서대문구 연세로 27-1,023256633,37.5581143,126.9366694
4,모범음식점,10024,KRW,서울특별시,용산구,엔그릴 N서울타워점,서울특별시 용산구 용산동2가 1-3,서울특별시 용산구 남산공원길 105,0234559297,37.5511225,126.9878678
...,...,...,...,...,...,...,...,...,...,...,...
48560,모범음식점,1186128,KRW,광주광역시,광산구,용우동 첨단점,광주광역시 광산구 쌍암동 687-2,광주광역시 광산구 첨단중앙로182번길 76,0629723377,35.2216073,126.8495768
48561,모범음식점,1198890,KRW,서울특별시,중랑구,씨유 망우씨너스빌점,서울특별시 중랑구 망우동 495,서울특별시 중랑구 용마산로115길 109,024920707,37.5974663,127.0941603
48562,모범음식점,1201448,KRW,충청북도,청주시,씨유 오창양청전원점,충청북도 청주시 청원구 오창읍 양청리 736-1,충청북도 청주시 청원구 오창읍 양청택지1길 49-2,0432177900,36.7201347,127.4213936
48563,모범음식점,1202664,KRW,경상북도,경산시,이마트24 경산계양타운점,경상북도 경산시 계양동 660-13,경상북도 경산시 원효로26길 8,0538110022,35.8219185,128.7505071


In [1329]:
ids = ','.join(map(str, df['RSTRNT_ID'].values.tolist()))


In [1330]:
conn = live_db_conn()
curs = conn.cursor()

sql = """ 
        SELECT p.id, s.id , s.`name`,'' as LOCGOV_CRTFC_FLAG_NM, p.`name` , p.price ,
        GROUP_CONCAT(DISTINCT CASE WHEN pt.`type` = 'ingredient' THEN ptm.`name` END SEPARATOR ', ') as MAIN_INGREDIENT_DC,
        GROUP_CONCAT(DISTINCT CASE WHEN pt.`type` = 'cooking' THEN ptm.`name` END SEPARATOR ', ') as CKNG_STYLE_DC,
		  GROUP_CONCAT(DISTINCT CASE WHEN pt.`type` = 'sauce' THEN ptm.`name` END SEPARATOR ', ') as SOURC_DC,
		  GROUP_CONCAT(DISTINCT CASE WHEN pt.`type` = 'option' THEN ptm.`name` END SEPARATOR ', ') as OPTN_DC
		  FROM store s
        LEFT JOIN product p ON s.id = p.store_id
        LEFT JOIN product_tag pt ON p.id = pt.product_id
        LEFT JOIN product_tag_master ptm ON pt.product_tag_master_id = ptm.id
        WHERE s.id IN ({})
        AND p.id IS NOT NULL AND p.`name` NOT REGEXP '예약|금액|상품권|이용'
        GROUP BY 
        p.id, s.id , s.`name`, p.`name` , p.price;""".format(ids)
curs.execute(sql)
menu = pd.DataFrame(curs.fetchall())

In [1332]:
menu['price'][menu['price'] < 0] = None
# NaN 값을 포함한 행 제거
menu = menu.dropna()
menu.info()

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\1880560783.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  menu['price'][menu['price'] < 0] = None
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\1880560783.py:1: SettingWithCopyWarning: 


<class 'pandas.core.frame.DataFrame'>
Index: 50683 entries, 18 to 793482
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    50683 non-null  int64 
 1   s.id                  50683 non-null  int64 
 2   name                  50683 non-null  object
 3   LOCGOV_CRTFC_FLAG_NM  50683 non-null  object
 4   p.name                50683 non-null  object
 5   price                 50683 non-null  object
 6   MAIN_INGREDIENT_DC    50683 non-null  object
 7   CKNG_STYLE_DC         50683 non-null  object
 8   SOURC_DC              50683 non-null  object
 9   OPTN_DC               50683 non-null  object
dtypes: int64(2), object(8)
memory usage: 4.3+ MB


In [1333]:
final = pd.merge(menu,df[['RSTRNT_ID','LOCGOV_CRTFC_FLAG_NM']],left_on='s.id',right_on='RSTRNT_ID',how='left')
final

final['LOCGOV_CRTFC_FLAG_NM_x'] = final['LOCGOV_CRTFC_FLAG_NM_y']

# RSTRNT_ID와 LOCGOV_CRTFC_FLAG_NM_y 컬럼 삭제
final.drop(columns=['RSTRNT_ID', 'LOCGOV_CRTFC_FLAG_NM_y'], inplace=True)
final_result = final.values.tolist()
final_result

[[20040,
  10012,
  '명동정',
  '모범음식점',
  '궁중떡볶이',
  Decimal('15000.00'),
  '떡, 소고기, 야채',
  '떡볶이',
  '간장',
  '궁중'],
 [20080,
  10014,
  '봉추찜닭',
  '모범음식점',
  '뼈없는찜닭(중/한마리)',
  Decimal('38000.00'),
  '닭고기, 야채',
  '찜',
  '간장',
  '뼈없는'],
 [20345,
  10019,
  '대포찜닭',
  '안심식당',
  '대포찜닭(4인/순살/순한맛)',
  Decimal('44000.00'),
  '닭고기, 야채',
  '찜',
  '간장',
  '뼈없는'],
 [20346,
  10019,
  '대포찜닭',
  '안심식당',
  '대포찜닭(4인/순살/매콤한맛)',
  Decimal('44000.00'),
  '닭고기, 야채',
  '찜',
  '매운',
  '뼈없는'],
 [20347,
  10019,
  '대포찜닭',
  '안심식당',
  '대포찜닭(4인/순살/매운맛)',
  Decimal('44000.00'),
  '닭고기, 야채',
  '찜',
  '매운',
  '뼈없는'],
 [20348,
  10019,
  '대포찜닭',
  '안심식당',
  '구름찜닭(치즈추가)',
  Decimal('3000.00'),
  '닭고기, 야채, 치즈',
  '찜',
  '매운',
  '추가'],
 [20352,
  10019,
  '대포찜닭',
  '안심식당',
  '대포찜닭(3인/순살/매운맛)',
  Decimal('33000.00'),
  '닭고기, 야채',
  '찜',
  '매운',
  '뼈없는'],
 [20353,
  10019,
  '대포찜닭',
  '안심식당',
  '대포찜닭(3인/순살/매콤한맛)',
  Decimal('33000.00'),
  '닭고기, 야채',
  '찜',
  '매운',
  '뼈없는'],
 [20357,
  10019,
  '대포찜닭',
  '안심식당',
  '대포찜닭(2인/

In [1335]:
table_nm

'RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_202407'

In [1336]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_TURSM_LMK_CFR_RSTRNT_INFO

In [1298]:
data_key = 'RT_TURSM_LMK_CFR_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_TURSM_LMK_CFR_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_TURSM_LMK_CFR_RSTRNT_INFO의 최근 테이블은 RT_TURSM_LMK_CFR_RSTRNT_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [1295]:
conn = dev_db_conn('redtable01')
curs = conn.cursor()

sql = """
SELECT T3.NM, SUBSTRING_INDEX(T3.AREA1,' ',1) AS 'Area1', T4.SRC_ID AS 'Store ID', T1.RSTR_NM,
T1.RSTR_RDNMADR , T1.RSTR_LNNO_ADRES , T1.RSTR_TELNO_PLAIN , T1.RSTR_LA , T1.RSTR_LO , T2.RSTR_LDMARK_DIST
FROM redtable01.RSTR T1
LEFT JOIN redtable01.RSTR_LDMARK T2 ON T1.ID = T2.RSTR_ID
LEFT JOIN redtable01.LDMARK_MASTER T3 ON T2.LDMARK_MASTER_ID = T3.ID AND T3.`TYPE` = '관광지'
LEFT JOIN redtable01.RSTR_CONN T4 ON T1.ID = T4.RSTR_ID AND T4.SRC = 'live_db'
LEFT JOIN redtable01.RSTR_INDEX T5 ON T1.ID = T5.RSTR_ID
WHERE T5.IS_USE = 1 AND T5.IS_OPEN = 1 AND T3.NM IS NOT NULL AND T3.NM != '' AND T4.SRC_ID IS NOT NULL 
"""
curs.execute(sql)
df = pd.DataFrame(curs.fetchall())

curs.close()
conn.close()
#df['RSTR_REPRSNT_TELNO'] = df.iloc[:, 6].astype(str).str.replace('-', '')
df['RSTR_TELNO_PLAIN'] = df['RSTR_TELNO_PLAIN'].str.replace('-','')
df['RSTR_TELNO_PLAIN'] = df['RSTR_TELNO_PLAIN'].replace('None', '')
df = df[~df['NM'].str.contains('(주)')]
df.sort_values(by=['NM'],ignore_index=True,inplace=True)
final_result = df[['NM', 'Area1', 'Store ID', 'RSTR_NM', 'RSTR_RDNMADR', 'RSTR_LNNO_ADRES','RSTR_TELNO_PLAIN', 'RSTR_LA', 'RSTR_LO', 'RSTR_LDMARK_DIST']].values.tolist()

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\3346569128.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['NM'].str.contains('(주)')]


In [1297]:
df

,NM,Area1,Store ID,RSTR_NM,RSTR_RDNMADR,RSTR_LNNO_ADRES,RSTR_TELNO_PLAIN,RSTR_LA,RSTR_LO,RSTR_LDMARK_DIST
0,(구)강경노동조합,충청남도,1186650,장날짜장,충청남도 논산시 강경읍 대흥로34번길 16,충청남도 논산시 강경읍 대흥리 7-2,None,36.1578058,127.0140517,344.20142
1,(구)인천일본제58은행지점,인천광역시,991934,차이나타운 카페 가배차관,인천광역시 중구 차이나타운로 46-1,인천광역시 중구 북성동2가 10-2,None,37.4754227,126.6191226,347.24350
2,(구)인천일본제58은행지점,인천광역시,1181079,샘터카페,인천광역시 중구 전동로 7,인천광역시 중구 전동 31-5,None,37.4758486,126.6263726,551.10223
3,(구)인천일본제58은행지점,인천광역시,1182470,담소연,인천광역시 중구 신포로46번길 8-8,인천광역시 중구 내동 91,0327771023,37.4722268,126.6262953,439.22831
4,(구)인천일본제58은행지점,인천광역시,1184307,비오는 포장마차,인천광역시 중구 우현로39번길 12,인천광역시 중구 신포동 12-62,None,37.4709756,126.6266409,509.37279
...,...,...,...,...,...,...,...,...,...,...
26462,히흣 도예공방,충청남도,638021,마블스크림천안신부점,충청남도 천안시 동남구 먹거리10길 17,충청남도 천안시 동남구 신부동 460-7,0415568551,36.8179784,127.1557238,477.23385
26463,히흣 도예공방,충청남도,1181994,김밥파는1004씨이오,충청남도 천안시 동남구 대흥로 262,충청남도 천안시 동남구 대흥동 5-13,None,36.8113143,127.1488900,622.14685
26464,히흣 도예공방,충청남도,1182916,디어문(Dearmoon),충청남도 천안시 동남구 먹거리9길 21,충청남도 천안시 동남구 신부동 459-10,None,36.8176536,127.1561712,507.05720
26465,히흣 도예공방,충청남도,1186692,재뉴어리하우스,충청남도 천안시 동남구 복자1길 26,충청남도 천안시 동남구 성황동 35-1,None,36.8142234,127.1524766,326.47654


In [1296]:
table_nm,col_nm,col_cnt

('RT_TURSM_LMK_CFR_RSTRNT_INFO_202407',
 'TRRSRT_NM,LOCPLC_DC,RSTRNT_ID,RSTRNT_NM,RSTRNT_ROAD_NM_ADDR,RSTRNT_LNM_ADDR,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO,TRRSRT_RSTRNT_BTWN_DSTNC_VALUE',
 '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s')

In [11]:
conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


In [20]:
table_nm

'RT_TURSM_LMK_CFR_RSTRNT_INFO_202309'

### RT_PET_ACP_POSBL_RSTRNT_INFO

In [1225]:
data_key = 'RT_PET_ACP_POSBL_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
'''

RT_PET_ACP_POSBL_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_PET_ACP_POSBL_RSTRNT_INFO의 최근 테이블은 RT_PET_ACP_POSBL_RSTRNT_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\n'

In [1226]:
# 애완견 반입 가능 상점 RSTR ID 먼저 추출
with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()
        sql ="""
        SELECT 
            R.ID, 
            "" AS 'LABEL', 
            R.RSTR_NM, 
            R.RSTR_RDNMADR, 
            R.RSTR_LNNO_ADRES, 
            R.RSTR_TELNO_PLAIN, 
            '' AS 'MENU', 
            R.RSTR_LA, 
            R.RSTR_LO
        FROM 
            redtable01.RSTR R
            INNER JOIN redtable01.RSTR_OPTION RO ON R.ID = RO.RSTR_ID
            left JOIN redtable01.RSTR_INDEX RI ON R.ID = RI.RSTR_ID
            left JOIN redtable01.RSTR_LABEL RL ON RL.RSTR_ID = R.ID
            left JOIN redtable01.RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
            left JOIN redtable01.MENU M ON M.RSTR_ID = R.ID
        WHERE 
            RI.IS_USE = 1 
            AND RI.IS_OPEN = 1
            AND RO.PET_ENTRN_POSBL_YN = 'Y';
    
        """
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()

        sql ="""
        SELECT R.ID, GROUP_CONCAT(RLM.LABEL_NM)
        FROM redtable01.RSTR R , redtable01.RSTR_LABEL RL, redtable01.RSTR_LABEL_MASTER RLM
        WHERE R.ID IN {}
        AND R.ID = RL.RSTR_ID AND RLM.ID = RL.LABEL_MASTER_ID
        GROUP BY R.ID
        """.format(tuple(df['ID']))

        curs.execute(sql)
        result =  pd.DataFrame(curs.fetchall())
        dl = dict(result.values)
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

id_list = tuple(df['ID']) 

with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()

        sql ="""
        SELECT R.ID, GROUP_CONCAT(DISTINCT M.MENU_NM SEPARATOR ',')
        FROM redtable01.RSTR R
        INNER JOIN redtable01.MENU M ON R.ID = M.RSTR_ID
        WHERE R.ID IN %s
        GROUP BY R.ID
        """

        curs.execute(sql, (id_list,))
        result =  pd.DataFrame(curs.fetchall())
        dm = dict(result.values)
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

# RSTR_ID에서 live_db의 store_id로 변환
conn = dev_db_conn('redtable01')
curs = conn.cursor()

sql = """
SELECT A.RSTR_ID, A.SRC_ID as store_id FROM (SELECT * FROM redtable01.RSTR_CONN where SRC='live_db') A
"""
curs.execute(sql)
target_store = pd.DataFrame(curs.fetchall())
df = pd.merge(df, target_store, how='inner',left_on='ID',right_on='RSTR_ID')

df['LABEL'] = df['ID'].apply(lambda x : dl.get(x))
df['MENU'] = df['ID'].apply(lambda x : dm.get(x))
df_result = df.drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])
df_result

쿼리 완료 
쿼리 작동 시간 : 0.9046766757965088
쿼리 완료 
쿼리 작동 시간 : 0.146636962890625
쿼리 완료 
쿼리 작동 시간 : 0.31478118896484375


,ID,LABEL,RSTR_NM,RSTR_RDNMADR,RSTR_LNNO_ADRES,RSTR_TELNO_PLAIN,MENU,RSTR_LA,RSTR_LO,RSTR_ID,store_id
0,1727,기타,카페 구할구,서울특별시 중구 퇴계로20길 73,서울특별시 중구 남산동2가 24-13,0237895953,"감자튀김,구할구 올데이 브런치,구할구커피(차가움),누텔라 초코 크로플,다방라떼,디카...",37.5582098,126.9870087,1727,40317
44,4369,"한식,일식",생선횟집,경상남도 남해군 설천면 노량로183번길 22,경상남도 남해군 설천면 노량리 355-14,055-862-2627,"낙지볶음(소),낙지볶음(중),돔,멍게비빔밥,모듬해물(대),모듬회(2,3인),모듬회(...",34.9414823,127.8751852,4369,19199
108,4793,양식,카페 바람종,부산광역시 기장군 일광읍 문오성길 487,부산광역시 기장군 일광읍 칠암리 55,051-728-0516,"보라빛레몬본능,아메리카노,아이스꿀자몽,자몽.생강.유자차,카라멜마끼아또,카페라떼",35.2975294,129.2596823,4793,199605
114,5857,한식,노마진포차,부산광역시 동구 조방로49번길 15,부산광역시 동구 범일동 830-256,010-9658-1028,저렴한 동네포차입니다.,35.1394010,129.0602758,5857,201510
115,8112,기타,이자카야 타비타비,부산광역시 부산진구 가야대로784번길 51,부산광역시 부산진구 부전동 517-33,0518194003,None,35.1551246,129.0573212,8112,203828
...,...,...,...,...,...,...,...,...,...,...,...
12213,936872,카페,헤브론153카페,경상남도 함양군 서하면 함양남서로 3760,경상남도 함양군 서하면 운곡리 707-5,070-7756-3026,None,35.6346305,127.6873292,936872,463482
12214,938516,카페,헬로팔로워즈,서울특별시 영등포구 도신로29길 12,서울특별시 영등포구 도림동 114-6,070-4355-5199,None,37.5096232,126.9049977,938516,39043
12215,938855,카페,파이드파이퍼,서울특별시 용산구 한강대로14길 8,서울특별시 용산구 한강로3가 65-156,0221352174,None,37.5249348,126.9646593,938855,127296
12216,953297,카페,공미학,서울특별시 서초구 서초대로48길 101,서울특별시 서초구 서초동 1564-1,0264629000,"달고나라떼,디저트 페어링,딸기듬뿍라떼,라떼는 메가톤바,레몬티,르베네피크 허브티(라벤...",37.4886222,127.0124697,953297,119876


In [1228]:
df_result.columns
df_result = df_result[['store_id','LABEL', 'RSTR_NM', 'RSTR_RDNMADR', 'RSTR_LNNO_ADRES','RSTR_TELNO_PLAIN', 'MENU', 'RSTR_LA', 'RSTR_LO']]
df_result['RSTR_TELNO_PLAIN'] = df_result['RSTR_TELNO_PLAIN'].str.replace('-','')
df_result

,store_id,LABEL,RSTR_NM,RSTR_RDNMADR,RSTR_LNNO_ADRES,RSTR_TELNO_PLAIN,MENU,RSTR_LA,RSTR_LO
0,40317,기타,카페 구할구,서울특별시 중구 퇴계로20길 73,서울특별시 중구 남산동2가 24-13,0237895953,"감자튀김,구할구 올데이 브런치,구할구커피(차가움),누텔라 초코 크로플,다방라떼,디카...",37.5582098,126.9870087
44,19199,"한식,일식",생선횟집,경상남도 남해군 설천면 노량로183번길 22,경상남도 남해군 설천면 노량리 355-14,0558622627,"낙지볶음(소),낙지볶음(중),돔,멍게비빔밥,모듬해물(대),모듬회(2,3인),모듬회(...",34.9414823,127.8751852
108,199605,양식,카페 바람종,부산광역시 기장군 일광읍 문오성길 487,부산광역시 기장군 일광읍 칠암리 55,0517280516,"보라빛레몬본능,아메리카노,아이스꿀자몽,자몽.생강.유자차,카라멜마끼아또,카페라떼",35.2975294,129.2596823
114,201510,한식,노마진포차,부산광역시 동구 조방로49번길 15,부산광역시 동구 범일동 830-256,01096581028,저렴한 동네포차입니다.,35.1394010,129.0602758
115,203828,기타,이자카야 타비타비,부산광역시 부산진구 가야대로784번길 51,부산광역시 부산진구 부전동 517-33,0518194003,None,35.1551246,129.0573212
...,...,...,...,...,...,...,...,...,...
12213,463482,카페,헤브론153카페,경상남도 함양군 서하면 함양남서로 3760,경상남도 함양군 서하면 운곡리 707-5,07077563026,None,35.6346305,127.6873292
12214,39043,카페,헬로팔로워즈,서울특별시 영등포구 도신로29길 12,서울특별시 영등포구 도림동 114-6,07043555199,None,37.5096232,126.9049977
12215,127296,카페,파이드파이퍼,서울특별시 용산구 한강대로14길 8,서울특별시 용산구 한강로3가 65-156,0221352174,None,37.5249348,126.9646593
12216,119876,카페,공미학,서울특별시 서초구 서초대로48길 101,서울특별시 서초구 서초동 1564-1,0264629000,"달고나라떼,디저트 페어링,딸기듬뿍라떼,라떼는 메가톤바,레몬티,르베네피크 허브티(라벤...",37.4886222,127.0124697


In [1229]:
final_result = df_result.values.tolist()

In [1230]:
# 수정후 재 업데이트시 이용
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_PET_ACP_POSBL_RSTRNT_INFO_2023_04")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_PET_ACP_POSBL_RSTRNT_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(col_cnt)
print(table_nm)

RSTRNT_ID,RSTRNT_CTGRY_NM,RSTRNT_NM,RSTRNT_ROAD_NM_ADDR,RSTRNT_LNM_ADDR,RSTRNT_TEL_NO,SLE_MENU_INFO_DC,RSTRNT_LA,RSTRNT_LO
%s,%s,%s,%s,%s,%s,%s,%s,%s
RT_PET_ACP_POSBL_RSTRNT_INFO_202407


In [1231]:
conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO_2024_07
온라인(비대면)주문서비스 제공 카페 베이커리 정보

In [1288]:
data_key = 'RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()'''


RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO의 최근 테이블을 확인합니다.
확인완료
RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO의 최근 테이블은 RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()'

In [1289]:
with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()

        sql ="""
        SELECT 
    R.id, 
    CONCAT(RLM.LABEL_NM, ',', R.BSNS_STATM_BZCND_NM, ',', R.SRVIC_CLS_NM) AS 'INDUTY_NM', 
    NR.name, 
    R.RSTR_RDNMADR, 
    R.RSTR_LNNO_ADRES, 
    R.RSTR_TELNO_PLAIN, 
    NR.delivery, 
    GROUP_CONCAT(M.MENU_NM SEPARATOR ', ') AS 'MENU_NM', 
    R.RSTR_LA, 
    R.RSTR_LO
FROM 
    redtable_raw_2022.naver_restaurant NR
    INNER JOIN redtable01.RSTR_CONN RC ON RC.SRC_ID = NR.id
    INNER JOIN redtable01.RSTR R ON R.ID = RC.RSTR_ID
    INNER JOIN redtable01.MENU M ON M.RSTR_ID = R.ID
    INNER JOIN redtable01.RSTR_LABEL RL ON RL.RSTR_ID = R.ID
    INNER JOIN redtable01.RSTR_LABEL_MASTER RLM ON RLM.ID = RL.LABEL_MASTER_ID
    INNER JOIN redtable01.RSTR_INDEX RI ON RI.RSTR_ID = R.ID
WHERE 
    RC.SRC = 'naver' 
    AND RI.IS_USE = 1 
    AND RI.IS_OPEN = 1
    AND RLM.LABEL_NM REGEXP '카페|베이커리'
    AND R.BSNS_STATM_BZCND_NM REGEXP '카페|베이커리|디저트|아이스크림|커피|도넛|빵|케이크'
    AND R.BSNS_STATM_BZCND_NM NOT REGEXP '라이브'
    AND NR.delivery IS NOT NULL
    AND NR.delivery != ''
GROUP BY 
    R.id, 
    CONCAT(RLM.LABEL_NM, ',', R.BSNS_STATM_BZCND_NM, ',', R.SRVIC_CLS_NM), 
    NR.name, 
    R.RSTR_RDNMADR, 
    R.RSTR_LNNO_ADRES, 
    R.RSTR_TELNO_PLAIN, 
    NR.delivery, 
    R.RSTR_LA, 
    R.RSTR_LO;
        """
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
        
df_result = df.drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])

쿼리 완료 
쿼리 작동 시간 : 3.53810977935791


In [1291]:
df_result.replace('',None,inplace=True)
df_result['RSTR_TELNO_PLAIN'] = df_result['RSTR_TELNO_PLAIN'].str.replace('-','')
df_result

,id,INDUTY_NM,name,RSTR_RDNMADR,RSTR_LNNO_ADRES,RSTR_TELNO_PLAIN,delivery,MENU_NM,RSTR_LA,RSTR_LO
0,11886,"카페,커피숍,기타",이디야커피 안성중대점,경기도 안성시 대덕면 대학3길 26,경기도 안성시 대덕면 내리 684-1,0316724035,배달의민족,"카페 라떼, 카페 아메리카노, 토피 넛 라떼, 바닐라 라떼",37.0021966,127.2258387
1,12652,"카페,아이스크림,기타",배스킨라빈스 당진신평점,충청남도 당진시 신평면 덕평로 1390,충청남도 당진시 신평면 거산리 165-4,0413633330,배달의민족,"싱글레귤러, 싱글킹, 더블쥬니어, 더블레귤러, 파인트, 쿼터, 패밀리, 하프갤론",36.8789363,126.7523890
2,45173,"카페,커피숍,기타",노브랜드버거 경희대점,서울특별시 동대문구 회기로 144,서울특별시 동대문구 회기동 60-26,029574910,배달의민족,"감자튀김(Medium), 그린샐러드(Combo), 그린샐러드(Single), 상하이...",37.5914488,127.0524338
3,61929,"카페,아이스크림,기타",배스킨라빈스 진해송학,경상남도 창원시 진해구 벚꽃로 71,경상남도 창원시 진해구 화천동 47-5,0555440760,배달의민족,"하프갤론, 패밀리, 쿼터, 파인트, 더블쥬니어, 싱글킹, 싱글레귤러, 더블레귤러",35.1518529,128.6635561
4,96401,"카페,아이스크림,기타",배스킨라빈스 세종조치원,세종특별자치시 조치원읍 으뜸길 222,세종특별자치시 조치원읍 원리 10-18,0448663121,배달의민족,"쿼터, 하프갤론, 패밀리, 파인트, 더블레귤러, 더블쥬니어, 싱글킹, 싱글레귤러",36.6015461,127.2969823
...,...,...,...,...,...,...,...,...,...,...
1252,970297,"카페,커피숍,기타",디저트39 한국교원대점,충청북도 청주시 흥덕구 강내면 태성탑연로 386,충청북도 청주시 흥덕구 강내면 탑연리 339,0432355500,배달의민족,"오리지널 도쿄롤, 커스타드 크로칸슈, 초코 크로와상",36.6172678,127.3559406
1253,970298,"카페,커피숍,기타",디저트39 한국교원대점,충청북도 청주시 흥덕구 강내면 태성탑연로 386,충청북도 청주시 흥덕구 강내면 탑연리 339,0432355500,배달의민족,"오리지널 도쿄롤, 커스타드 크로칸슈, 초코 크로와상",36.6172678,127.3559406
1254,970519,"카페,아이스크림,기타",배스킨라빈스 김해삼정점,경상남도 김해시 활천로36번길 27-9,경상남도 김해시 삼정동 196-14,050720939403,배달의민족,"파인트, 하프갤론, 패밀리, 쿼터, 더블쥬니어, 싱글킹, 싱글레귤러, 더블레귤러",35.2321860,128.8956086
1255,970826,"카페,커피숍,기타",에이비카페,서울특별시 강남구 강남대로102길 32,서울특별시 강남구 역삼동 618-15,025662052,배달의민족,"페퍼민트 레몬(ICED), 청포도 에이드(ICED), 페퍼민트 레몬(HOT), 와일...",37.5029048,127.0279383


In [1292]:
final_result = df_result.values.tolist()
conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_HALAL_MENU_INFO_2024_07

In [145]:
data_key = 'RT_HALAL_MENU_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())
orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]


RT_HALAL_MENU_INFO의 최근 테이블을 확인합니다.


'RT_HALAL_MENU_INFO_2023_02'

In [1060]:
orig_nm = 'RT_HALAL_MENU_INFO_2022_09'
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())
summary

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)
'''
print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()'''


확인완료
RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO의 최근 테이블은 RT_HALAL_MENU_INFO_2022_09입니다.


'\nprint(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()'

In [1069]:
with dev_db_conn('redtable_raw_2022') as conn:
    with conn.cursor() as curs:
        start = time.time()

        sql =""" select *
        from redtable_raw_2022.halar_source ;"""
        
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
df['store_id']

0      840855
1      844614
2      118840
3      119218
4      122384
        ...  
136     35363
137     35366
138    256396
139    816339
140    197487
Name: store_id, Length: 141, dtype: int64

In [1070]:
# halar_source 기반 진행

conn = live_db_conn()
curs = conn.cursor()

ids = ','.join(map(str, df['store_id'].values))

sql = '''
    SELECT DISTINCT 
    p.id, 
    p.`name`, 
    pt_en_translate.`name` AS en_name,
    GROUP_CONCAT(DISTINCT ptm.`name`) AS tag_names,
    GROUP_CONCAT(DISTINCT pt_zh_hans.`name`) AS zh_hans_names,
    GROUP_CONCAT(DISTINCT pt_ja.`name`) AS ja_names,
    GROUP_CONCAT(DISTINCT pt_en.`name`) AS en_tag_names,
    p.price,
    'KRW' AS 'CRNCY', NULL AS 'IMAGE', 'KOR' AS 'COUNTRY',
	  SUBSTRING_INDEX(s.address, ' ', 1) AS 'area_1' , s.id
FROM 
    product p
LEFT JOIN store s ON p.store_id = s.id
LEFT JOIN (  SELECT product_id,`name`
    FROM product_translate
    WHERE  lang = 'en'
) pt_en_translate ON pt_en_translate.product_id = p.id
LEFT JOIN product_tag pt ON p.id = pt.product_id
LEFT JOIN product_tag_master ptm ON pt.product_tag_master_id = ptm.id
LEFT JOIN (
    SELECT  product_tag_master_id,`name`
    FROM  product_tag_translate
    WHERE lang = 'zh-Hans'
) pt_zh_hans ON pt_zh_hans.product_tag_master_id = ptm.id
LEFT JOIN (
    SELECT  product_tag_master_id, `name`
    FROM product_tag_translate
    WHERE lang = 'ja'
) pt_ja ON pt_ja.product_tag_master_id = ptm.id
LEFT JOIN (
    SELECT 
        product_tag_master_id, 
        `name`
    FROM 
        product_tag_translate
    WHERE 
        lang = 'en'
) pt_en ON pt_en.product_tag_master_id = ptm.id
WHERE 
    p.`name` NOT REGEXP '맥주|위스키|와인|소주|술|칵테일|막걸리|사케'
    AND p.store_id IN ({})
GROUP BY 
    p.id, 
    p.`name`, 
    pt_en_translate.`name`;'''.format(ids)

curs.execute(sql)
df =  pd.DataFrame(curs.fetchall())
df

,id,name,en_name,tag_names,zh_hans_names,ja_names,en_tag_names,price,CRNCY,IMAGE,COUNTRY,area_1,s.id
0,40424,간장게장 정식(1인),Soy Sauce Marinated Crab Set Menu(For 1),"간장,간장게장,게","蟹,酱油,酱蟹","カニ,カンジャン,カンジャンケジャン","Crab,Soy Sauce,Soy Sauce Marinated Crab",15000.00,KRW,None,KOR,제주특별자치도,10753
1,40425,알뜰정식(2인),Budget Set Menu(For 2),"광어,생선회,정식","套餐,左口鱼,生鱼片","ヒラメ,刺身,定食","Flatfish,Set Menu,Sliced Raw Fish",39000.00,KRW,None,KOR,제주특별자치도,10753
2,40426,바다 스페셜(小),Bada Special(S),None,None,None,None,190000.00,KRW,None,KOR,제주특별자치도,10753
3,40569,바다 스페셜(大),Bada Special(Large),None,None,None,None,300000.00,KRW,None,KOR,제주특별자치도,10753
4,40570,바다 스페셜(中),Bada Special(Medium),None,None,None,None,250000.00,KRW,None,KOR,제주특별자치도,10753
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329,4446554,Seasonal Special Menu(주중 디너 1부/17:30-19:30),Seasonal Special Menu(Weekday Dinner Part 1/17...,None,None,None,None,172000.00,KRW,None,KOR,서울특별시,10757
3330,4446555,Seasonal Special Menu(주중 디너 2부/20:00-22:00),Seasonal Special Menu(Weekday Dinner Part 2/20...,None,None,None,None,172000.00,KRW,None,KOR,서울특별시,10757
3331,4446556,Seasonal Special Menu(주말 및 공휴일 디너1부/17:00-19:00),Seasonal Special Menu(Weekend and Holiday Dinn...,None,None,None,None,172000.00,KRW,None,KOR,서울특별시,10757
3332,4446557,Seasonal Special Menu(주말 및 공휴일 디너2부/19:30-21:30),Seasonal Special Menu(Weekend and Holiday Dinn...,None,None,None,None,172000.00,KRW,None,KOR,서울특별시,10757


In [1086]:
pattern = '맥주|위스키|와인|소주|술|칵테일|막걸리|사케|돼지|소|주말|세트|곱창|예약|뷔|육회|고기|약주|과실주|전통주|탕수육|한우|수육|우삼겹|삼겹살|삼겹|갈비|찜|차돌박이|돈까스|스끼야끼|치킨|족보|보쌈|백세주|복분자|콜라|사이다|공기밥|음료수|산사춘|매취순|카스|클라우드|매화수|스프라이트|화랑|음료|아메리카노'
df = df.fillna('')
filtered_df = df[
    ~df['tag_names'].str.contains(pattern) &
    ~df['name'].str.contains(pattern)]
filtered_df.drop_duplicates(subset='id',inplace=True)
filtered_df.replace('',None,inplace=True)


C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\678807534.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop_duplicates(subset='id',inplace=True)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\678807534.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.replace('',None,inplace=True)


In [1087]:
filtered_df[filtered_df['id'] == 2882003]

,id,name,en_name,tag_names,zh_hans_names,ja_names,en_tag_names,price,CRNCY,IMAGE,COUNTRY,area_1,s.id


In [1082]:
# 수정후 재 업데이트시 이용
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_HALAL_MENU_INFO_2022_09")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_HALAL_MENU_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(col_cnt)
print(table_nm)

MENU_ID,MENU_KLANG_NM,MENU_ROMAN_NM,MENU_KLANG_DC,MENU_CHNLNG_DC,MENU_JLANG_DC,MENU_ENGL_DC,MENU_PRC,CRNCY_CD,MENU_IMAGE_URL,COUNTRY_NM,CTY_NM,RSTRNT_ID
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
RT_HALAL_MENU_INFO_202407


In [1088]:
final_result = filtered_df.values.tolist()
conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


###  RT_CN_MENU_INFO  ---> 기존 데이터 재사용 권장

In [12]:
data_key = 'RT_CN_MENU_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()


RT_CN_MENU_INFO의 최근 테이블을 확인합니다.
확인완료
RT_CN_MENU_INFO의 최근 테이블은 RT_CN_MENU_INFO_2022_09입니다.
테이블을 생성합니다.


In [ ]:
with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:
        start = time.time()

        sql ="""
        SELECT M.ID
        , (CASE WHEN LANG_NM = 'CN' THEN MT.MENU_NM END) AS '간체'
        , M.MENU_NM '한'
        , (CASE WHEN LANG_NM = 'JP' THEN MT.MENU_NM END) AS '일'
        , (CASE WHEN LANG_NM = 'EN' THEN MT.MENU_NM END) AS '영'
        #, (CASE WHEN LANG_NM = 'TW' THEN MT.MENU_NM END) AS '번체'
        , M.MENU_PRICE '메뉴가격'
        , "KRW" as '유통화폐코드'
        , "KOREA" as "국가명"
        , R.AREA_NM_1 as "도시명"
        , R.ID "식당ID"
        FROM redtable01.MENU M, redtable01.MENU_TRANSLATE MT, redtable01.RSTR R, redtable01.RSTR_INDEX RI
        WHERE MT.MENU_ID = M.ID AND R.ID = M.RSTR_ID AND RI.RSTR_ID = R.ID
        AND RI.IS_USE = 1 AND RI.IS_OPEN = 1
        """
        curs.execute(sql)
        result = pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 


### RT_FRNR_RESVE_POSBL_RSTRNT_INFO
외국인 예약 가능 매장

In [14]:
data_key = 'RT_FRNR_RESVE_POSBL_RSTRNT_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
'''

RT_FRNR_RESVE_POSBL_RSTRNT_INFO의 최근 테이블을 확인합니다.
확인완료
RT_FRNR_RESVE_POSBL_RSTRNT_INFO의 최근 테이블은 RT_FRNR_RESVE_POSBL_RSTRNT_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\n'

In [23]:
with live_db_conn() as conn:
    with conn.cursor() as curs:
        start = time.time()
        sql ="""
        SELECT T1.id AS 'RSTRNT_ID', 'ENG' AS 'MOP_LANG_NM', GROUP_CONCAT(DISTINCT T4.name) AS 'RSTRNT_CTGRY_NM', 
        T1.name AS 'RSTRNT_NM', T2.name AS 'RSTRNT_ENG_NM', T1.address_road AS 'RSTRNT_ROAD_NM_ADDR', T2.address_road AS 'RSTRNT_ENG_RDNMADR_NM',
        T1.address AS 'RSTRNT_LNM_ADDR', T2.address AS 'RSTRNT_ENG_LNM_ADDR', T1.telephone_plain AS 'RSTRNT_TEL_NO', T1.homepage AS 'HMPG_URL',
        T1.latitude AS 'RSTRNT_LA', T1.longitude AS 'RSTRNT_LO'
        FROM redtable2021.store T1
        INNER JOIN redtable2021.store_translate T2 ON T1.id = T2.store_id
        LEFT JOIN redtable2021.store_label T3 ON T2.store_id = T3.store_id
        LEFT JOIN redtable2021.store_label_master T4 ON T3.store_label_master_id = T4.id 
            AND T4.name NOT LIKE '%>%' AND T4.name NOT LIKE '%명동상인협의회%' AND T4.name NOT LIKE '%must eat%'
        WHERE T1.`status` = 'normal' AND (T1.sub_category IS NULL OR T1.sub_category = 1)
            AND T1.id NOT IN (10001, 10002, 10003, 10004, 10005, 10068, 10069, 948544, 948548, 948549, 948550, 948551, 948553, 1074709, 1165173, 1631449)
            AND T1.name NOT LIKE '%?%' AND T2.lang = 'en'
            AND LENGTH(T1.telephone_plain) < 11
            AND (T1.telephone_plain LIKE '0%' or T1.telephone_plain LIKE '1%')
            AND (T1.id IN (SELECT store_id FROM redtable2021.reserve_with_google WHERE `status` = 'open')
            OR T1.id IN (SELECT T1.store_id
            FROM redtable2021.store_contract T1
            INNER JOIN redtable2021.store T2 ON T1.store_id = T2.id
            WHERE T1.`status` = 'normal' AND T1.detail_status = 'formal' AND T2.category = 'food' AND T2.`status` = 'normal'))
        GROUP BY T1.id;
        """
        #
#        INNER JOIN redtable_raw_2022.halar_source H2 ON R.ID = H2.RSTR_ID 제외
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

쿼리 완료 
쿼리 작동 시간 : 2.2990665435791016


In [25]:
final_result = df.values.tolist()
final_result

[[10012,
  'ENG',
  '한식',
  '명동정',
  'Myeongdongjeong',
  '서울특별시 중구 삼일대로 299',
  '299, Samil-daero, Jung-gu, Seoul',
  '서울특별시 중구 충무로2가 60-3',
  '60-3, Chungmuro 2(i)-ga, Jung-gu, Seoul',
  '0237895130',
  'http://www.hanjeongik.co.kr/,http://myongdongjeong.modoo.at/',
  Decimal('37.56157041880520'),
  Decimal('126.98847133150600')],
 [10014,
  'ENG',
  '한식',
  '봉추찜닭',
  'BONGCHU BRAISED CHICKEN',
  '서울특별시 중구 명동8나길 47',
  '47, Myeongdong 8na-gil, Jung-gu, Seoul',
  '서울특별시 중구 충무로1가 25-8',
  '25-8, Chungmuro 1(il)-ga, Jung-gu, Seoul',
  '0237899381',
  'http://www.bongchu.com/',
  Decimal('37.56106356160560'),
  Decimal('126.98227707999800')],
 [10016,
  'ENG',
  '한식',
  '명동한우방',
  'Myeongdong Hanu Bang',
  '서울특별시 중구 명동9길 37-5',
  '37-5, Myeongdong 9-gil, Jung-gu, Seoul',
  '서울특별시 중구 을지로2가 199-56',
  '199-56, Euljiro 2(i)-ga, Jung-gu, Seoul',
  '0222678132',
  None,
  Decimal('37.56538560898970'),
  Decimal('126.98393868038600')],
 [10018,
  'ENG',
  '한식',
  '삼호복집',
  'Samhobokjip',
  '서울

In [20]:
col_nm

'RSTRNT_ID,MOP_LANG_NM,RSTRNT_CTGRY_NM,RSTRNT_NM,RSTRNT_ENG_NM,RSTRNT_ROAD_NM_ADDR,RSTRNT_ENG_RDNMADR_NM,RSTRNT_LNM_ADDR,RSTRNT_ENG_LNM_ADDR,RSTRNT_TEL_NO,HMPG_URL,RSTRNT_LA,RSTRNT_LO'

In [26]:
print('전체 데이터를 새 테이블에 insert합니다.')
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')



전체 데이터를 새 테이블에 insert합니다.
insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_GLOBAL_OTA_FDTURSM_INFO

In [912]:
data_key = 'RT_GLOBAL_OTA_FDTURSM_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {}""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

table_nm = '{}_{}'.format(data_key,date_string)
'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')
'''

RT_GLOBAL_OTA_FDTURSM_INFO의 최근 테이블을 확인합니다.
확인완료
RT_GLOBAL_OTA_FDTURSM_INFO의 최근 테이블은 RT_GLOBAL_OTA_FDTURSM_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')\n'

In [927]:
conn = live_db_conn()
curs = conn.cursor()

sql = """ 
        SELECT o2.`code`, p.id, p.`name`, p.price, '' AS CRNCY_CD,
(CASE WHEN s.branch_name IS NOT NULL THEN CONCAT(s.`name`, '(', s.branch_name, ')')
 ELSE s.`name` 
 END) AS RSTRNT_NM, s.address
FROM ota_product o
INNER JOIN product p ON o.product_id = p.id
LEFT JOIN ota o2 ON o2.id = o.ota_id
LEFT JOIN store s ON p.store_id = s.id 
where o2.`code` != 'test' and s.address is not null and s.`name` not like '%테스트%'
and s.`name` not like '%CARD%';"""

curs.execute(sql)
df = pd.DataFrame(curs.fetchall())
df

,code,id,name,price,CRNCY_CD,RSTRNT_NM,address
0,kkday,20761,만족오향족발(중),36000.00,,만족오향족발(시청점),서울특별시 중구 태평로2가 318-1
1,kkday,526407,백록,240000.00,,무궁화(롯데호텔 서울),서울특별시 중구 소공동 1
2,kkday,23722,마늘족발(중),41000.00,,미쓰족발,서울특별시 중구 을지로2가 199-45
3,kkday,526171,"2인 코스 (17, 18, 19시)",262000.00,,더플레이스다이닝(N서울타워점),서울특별시 용산구 용산동2가 1-3
4,kkday,23720,반반족발,45000.00,,미쓰족발,서울특별시 중구 을지로2가 199-45
...,...,...,...,...,...,...,...
8994,wemakeprice_plus,4447744,날치알 볶음밥,3000.00,,애월찜,제주특별자치도 제주시 애월읍 신엄리 2807-6
8995,wemakeprice_plus,4447753,코코넛 쉬림프(인기메뉴),19000.00,,애월로맨틱새우,제주특별자치도 제주시 애월읍 곽지리 1584-5
8996,wemakeprice_plus,4447752,레드빅뱅 쉬림프(인기메뉴),29000.00,,애월로맨틱새우,제주특별자치도 제주시 애월읍 곽지리 1584-5
8997,wemakeprice_plus,4447756,로맨틱 BBQ 쉬림프(인기메뉴),15000.00,,애월로맨틱새우,제주특별자치도 제주시 애월읍 곽지리 1584-5


In [928]:
df['CRNCY_CD'] = 'KRW'
df['s2.address'] = 'KOR'
df['ct'] = df['address'].apply(lambda x : x.split(' ')[0] if x is not None else None)
df.fillna('',inplace=True)
final_result = df.values.tolist()
df

,code,id,name,price,CRNCY_CD,RSTRNT_NM,address,s2.address,ct
0,kkday,20761,만족오향족발(중),36000.00,KRW,만족오향족발(시청점),서울특별시 중구 태평로2가 318-1,KOR,서울특별시
1,kkday,526407,백록,240000.00,KRW,무궁화(롯데호텔 서울),서울특별시 중구 소공동 1,KOR,서울특별시
2,kkday,23722,마늘족발(중),41000.00,KRW,미쓰족발,서울특별시 중구 을지로2가 199-45,KOR,서울특별시
3,kkday,526171,"2인 코스 (17, 18, 19시)",262000.00,KRW,더플레이스다이닝(N서울타워점),서울특별시 용산구 용산동2가 1-3,KOR,서울특별시
4,kkday,23720,반반족발,45000.00,KRW,미쓰족발,서울특별시 중구 을지로2가 199-45,KOR,서울특별시
...,...,...,...,...,...,...,...,...,...
8994,wemakeprice_plus,4447744,날치알 볶음밥,3000.00,KRW,애월찜,제주특별자치도 제주시 애월읍 신엄리 2807-6,KOR,제주특별자치도
8995,wemakeprice_plus,4447753,코코넛 쉬림프(인기메뉴),19000.00,KRW,애월로맨틱새우,제주특별자치도 제주시 애월읍 곽지리 1584-5,KOR,제주특별자치도
8996,wemakeprice_plus,4447752,레드빅뱅 쉬림프(인기메뉴),29000.00,KRW,애월로맨틱새우,제주특별자치도 제주시 애월읍 곽지리 1584-5,KOR,제주특별자치도
8997,wemakeprice_plus,4447756,로맨틱 BBQ 쉬림프(인기메뉴),15000.00,KRW,애월로맨틱새우,제주특별자치도 제주시 애월읍 곽지리 1584-5,KOR,제주특별자치도


In [930]:
table_nm

'RT_GLOBAL_OTA_FDTURSM_INFO_202407'

In [931]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_SALE_INFO


In [964]:
data_key = 'RT_RESTAURANT_SALE_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)



RT_RESTAURANT_SALE_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_SALE_INFO의 최근 테이블은 RT_RESTAURANT_SALE_INFO_202407입니다.


In [ ]:
print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

In [965]:
conn  = live_db_conn()
curs  = conn.cursor()
start = time.time()

sql ="""
SELECT 
    TRIM(S.address) AS address,
    O.store_id, 
    S.name, 
        COALESCE(
        ST.name, 
        S.`name`
    ) AS MENU_USER_NM,
    P.`name` AS product_name,
    COALESCE(
        PT.name, 
        P.`name`
    ) AS MENU_USER_NM,
    '' as CRNCY,
    P.price,
    LEFT(O.order_at, 10) AS 'USE_DT',
    RIGHT(O.order_at, 8) AS 'USE_TM',
    O.order_at AS 'BUY_DT'
FROM `order` O
LEFT JOIN order_product O2 ON O2.order_id = O.id
LEFT JOIN ( select store_id, address from store_translate where lang ='en')
ST2 ON ST2.store_id = O.store_id 
INNER JOIN redtable2021.product P ON P.id = O2.product_id
INNER JOIN redtable2021.store S ON S.id = O.store_id
LEFT JOIN (
    SELECT 
        product_id, 
        `name`,
        lang
    FROM redtable2021.product_translate
    WHERE lang IN ('en', 'ja', 'zh-Hans', 'zh-Hant')
) PT ON PT.product_id = P.id AND PT.lang = O.order_lang
LEFT JOIN (
    SELECT 
        store_id, 
        `name`,
        lang
    FROM redtable2021.store_translate
    WHERE lang IN ('en', 'ja', 'zh-Hans', 'zh-Hant')
) ST ON ST.store_id = O.store_id AND ST.lang = O.order_lang
WHERE 
    O.name NOT REGEXP '[가-힣]'
    AND O.name NOT REGEXP 'test|테스트'
    AND P.name NOT REGEXP 'test|테스트'
    AND O.title NOT REGEXP 'test|테스트|V CARD|Reservation';
"""
curs.execute(sql)
order2 =  pd.DataFrame(curs.fetchall())
print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
curs.close()
conn.close()
order2

쿼리 완료 
쿼리 작동 시간 : 1.7056913375854492


,address,store_id,name,MENU_USER_NM,product_name,.MENU_USER_NM,CRNCY,price,USE_DT,USE_TM,BUY_DT
0,서울특별시 중구 을지로6가 18-37,10071,순미네 행복게장,Sunmine Haengbok Gejang,간장게장알베기(1kg),Soy Sauce Marinated Crab with Roe(1kg),,96000.00,2021-09-27,01:58:03,2021-09-27 01:58:03
1,전라남도 보성군 보성읍 원봉리 2-1,10612,특미관,Teungmigwan,녹차한우떡갈비(1인기준),Grilled Galbi Patties with Green Tea and Korea...,,24000.00,2021-09-28,05:50:15,2021-09-28 05:50:15
2,전라남도 순천시 금곡동 151,10711,옥천귀뚜라미,Okcheongwitturami,동파육덮밥(1인분),Red Braised Pork Belly with Rice(For 1),,9000.00,2021-10-01,06:11:58,2021-10-01 06:11:58
3,전라남도 순천시 금곡동 151,10711,옥천귀뚜라미,Okcheongwitturami,동파육덮밥(1인분),Red Braised Pork Belly with Rice(For 1),,9000.00,2021-10-01,06:12:12,2021-10-01 06:12:12
4,전라남도 영암군 삼호읍 용앙리 1685-15,10742,뉴 장충동왕족발보쌈,Nyu Jangchungdongwang Jokbal Bossam,족발(대),Braised Pigs' Feet(Large),,35000.00,2021-10-06,03:14:14,2021-10-06 03:14:14
...,...,...,...,...,...,...,...,...,...,...,...
23496,서울특별시 중구 을지로2가 199-45,10080,미쓰족발,Myth Jokbal,마늘족발(중),Braised Pigs' Feet with Garlic(Medium),,41000.00,2024-07-15,03:56:00,2024-07-15 03:56:00
23497,서울특별시 중구 충무로1가 23-10,13037,황금목장,黃金牧場,A세트(2인),Aセット(2人),,65000.00,2024-07-15,07:02:54,2024-07-15 07:02:54
23498,서울특별시 영등포구 여의도동 85-1,35470,애슐리퀸즈&이랜드크루즈,ASHLEY QUEENS & E-land Cruise,[여의도]한강투어 크루즈(16:00)+애슐리 주중런치(13:00),[Yeouido]Hangangtueo Cruise(16:00)+Aesyulli Ju...,,33900.00,2024-07-15,08:05:00,2024-07-15 08:05:00
23499,서울특별시 영등포구 여의도동 85-1,35470,애슐리퀸즈&이랜드크루즈,ASHLEY QUEENS & E-land Cruise,애슐리퀸즈 평일 런치(대인/1인),Weekday Lunch,,19900.00,2024-07-15,08:05:00,2024-07-15 08:05:00


In [967]:
order2['address'] = order2['address'].apply(lambda x : x.split(' ')[0] if x is not None else '')
order2['CRNCY'] = 'KRW'
order2

,address,store_id,name,MENU_USER_NM,product_name,.MENU_USER_NM,CRNCY,price,USE_DT,USE_TM,BUY_DT
0,서울특별시,10071,순미네 행복게장,Sunmine Haengbok Gejang,간장게장알베기(1kg),Soy Sauce Marinated Crab with Roe(1kg),KRW,96000.00,2021-09-27,01:58:03,2021-09-27 01:58:03
1,전라남도,10612,특미관,Teungmigwan,녹차한우떡갈비(1인기준),Grilled Galbi Patties with Green Tea and Korea...,KRW,24000.00,2021-09-28,05:50:15,2021-09-28 05:50:15
2,전라남도,10711,옥천귀뚜라미,Okcheongwitturami,동파육덮밥(1인분),Red Braised Pork Belly with Rice(For 1),KRW,9000.00,2021-10-01,06:11:58,2021-10-01 06:11:58
3,전라남도,10711,옥천귀뚜라미,Okcheongwitturami,동파육덮밥(1인분),Red Braised Pork Belly with Rice(For 1),KRW,9000.00,2021-10-01,06:12:12,2021-10-01 06:12:12
4,전라남도,10742,뉴 장충동왕족발보쌈,Nyu Jangchungdongwang Jokbal Bossam,족발(대),Braised Pigs' Feet(Large),KRW,35000.00,2021-10-06,03:14:14,2021-10-06 03:14:14
...,...,...,...,...,...,...,...,...,...,...,...
23496,서울특별시,10080,미쓰족발,Myth Jokbal,마늘족발(중),Braised Pigs' Feet with Garlic(Medium),KRW,41000.00,2024-07-15,03:56:00,2024-07-15 03:56:00
23497,서울특별시,13037,황금목장,黃金牧場,A세트(2인),Aセット(2人),KRW,65000.00,2024-07-15,07:02:54,2024-07-15 07:02:54
23498,서울특별시,35470,애슐리퀸즈&이랜드크루즈,ASHLEY QUEENS & E-land Cruise,[여의도]한강투어 크루즈(16:00)+애슐리 주중런치(13:00),[Yeouido]Hangangtueo Cruise(16:00)+Aesyulli Ju...,KRW,33900.00,2024-07-15,08:05:00,2024-07-15 08:05:00
23499,서울특별시,35470,애슐리퀸즈&이랜드크루즈,ASHLEY QUEENS & E-land Cruise,애슐리퀸즈 평일 런치(대인/1인),Weekday Lunch,KRW,19900.00,2024-07-15,08:05:00,2024-07-15 08:05:00


In [968]:
df_result = order2.drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])
#df_result['address'].replace(' ','',inplace=True)

In [970]:
col_nm

'CITY_NM,RSTRNT_ID,RSTRNT_LOCAL_NM,RSTRNT_USER_NM,MENU_LOCAL_NM,MENU_USER_NM,CRNCY,SALE_AM,USE_DT,USE_TM,BUY_DT'

In [971]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

final_result = df_result.values.tolist()

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_COUPON_KO_INFO
1. 쿠폰 title 추출
2. 가장 많은것들 위주로 번역요청 진행 (4개 테이블 동시 번역 요청 진행)
3. restaurant_id 와 store_id 매핑후 store_id 데이터에 붙히기

In [364]:
conn  = dev_db_conn('redtable_raw_2022')
curs  = conn.cursor()
start = time.time()

sql ="""
SELECT * 
FROM naver_coupon ;
"""
curs.execute(sql)
naver_coupon =  pd.DataFrame(curs.fetchall())
print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 
curs.close()
conn.close()
naver_coupon

쿼리 완료 
쿼리 작동 시간 : 0.28849220275878906


,promotion_id,coupon_id,restaurant_id,title,image
0,323,426,1710029265,"1,000원 할인 쿠폰",
1,333,436,1138224421,리뷰이벤트 서비스 쿠폰,
2,335,438,1138224421,"2,000원 할인 쿠폰",
3,354,457,1851129272,삼겹살130g증정 쿠폰,https://ldb-phinf.pstatic.net/20220906_68/1662...
4,386,489,1716243727,삼겹살130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_16/1662...
...,...,...,...,...,...
13579,201513,202348,1254193590,10% 할인 쿠폰,
13580,201526,202361,1053226058,고래나무 버터쿠키 1EA 쿠폰,https://ldb-phinf.pstatic.net/20230822_98/1692...
13581,201528,202363,1282450325,1인분 쿠폰,
13582,201529,202364,1053226058,10% 할인 쿠폰,


In [365]:
import re
def korean(text):
    pattern = re.compile('[^ ㄱ-ㅣ가-힣]+')
    return pattern.sub('',text)

naver_coupon['title_copy'] = naver_coupon['title'].apply(korean)
naver_coupon

,promotion_id,coupon_id,restaurant_id,title,image,title_copy
0,323,426,1710029265,"1,000원 할인 쿠폰",,원 할인 쿠폰
1,333,436,1138224421,리뷰이벤트 서비스 쿠폰,,리뷰이벤트 서비스 쿠폰
2,335,438,1138224421,"2,000원 할인 쿠폰",,원 할인 쿠폰
3,354,457,1851129272,삼겹살130g증정 쿠폰,https://ldb-phinf.pstatic.net/20220906_68/1662...,삼겹살증정 쿠폰
4,386,489,1716243727,삼겹살130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_16/1662...,삼겹살 제공 쿠폰
...,...,...,...,...,...,...
13579,201513,202348,1254193590,10% 할인 쿠폰,,할인 쿠폰
13580,201526,202361,1053226058,고래나무 버터쿠키 1EA 쿠폰,https://ldb-phinf.pstatic.net/20230822_98/1692...,고래나무 버터쿠키 쿠폰
13581,201528,202363,1282450325,1인분 쿠폰,,인분 쿠폰
13582,201529,202364,1053226058,10% 할인 쿠폰,,할인 쿠폰


In [258]:
a = naver_coupon['title_copy'].value_counts()[:150]
c = pd.DataFrame(a)
c.to_excel("번역요청.xlsx")

#### 번역완료 데이터 이용해 매핑 
(100개정도 단어 추가 추출 -> 번역요청(요청시 숫자는 골뱅이로 전체 바꿈)-> 번역완료 -> 완료데이터에 골뱅이 숫자 넣기 -> 100개 번역데이터 기반으로 테이블 업데이트 진행)


In [341]:
# 변역완료후 진행 과정
refer = pd.read_excel("번역요청.xlsx",sheet_name='Sheet2')
base = pd.read_excel("번역요청.xlsx",sheet_name='Sheet1')
testa=pd.merge(base,refer,left_on='title_copy',right_on='요청단어',how='left')
base

,title_copy,count,Unnamed: 2,title_copy.1
0,쿠폰,4,NaN,NaN
1,쿠폰,7,NaN,쿠폰
2,전화 쿠폰,6,NaN,전화 쿠폰
3,쿠폰,14,NaN,쿠폰
4,할인 쿠폰,1783,NaN,할인 쿠폰
...,...,...,...,...
145,하이볼 쿠폰,16,NaN,하이볼 쿠폰
146,하이볼 한잔 무료 쿠폰,3,NaN,하이볼 한잔 무료 쿠폰
147,하이볼잔 쿠폰,4,NaN,NaN
148,한우육회 쿠폰,5,NaN,한우육회 쿠폰


In [354]:
refer = pd.read_excel("번역요청2.xlsx")
refer
# 번역 합치기
##a = naver_coupon['title_copy'].value_counts()[:150]
#c = pd.DataFrame(a)

,Unnamed: 0,title_copy,title_copy.1,요청단어,EN,JP,CN,TW
0,0,쿠폰,NaN,쿠폰,Coupons,クーポン,优惠券,優惠券
1,1,쿠폰,쿠폰,쿠폰,Coupons,クーポン,优惠券,優惠券
2,2,전화 쿠폰,전화 쿠폰,전화 쿠폰,Phone Coupon,電話クーポン,电话优惠券,電話優惠券
3,3,쿠폰,쿠폰,쿠폰,Coupons,クーポン,优惠券,優惠券
4,4,할인 쿠폰,할인 쿠폰,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券
...,...,...,...,...,...,...,...,...
147,147,하이볼 쿠폰,하이볼 쿠폰,하이볼 쿠폰,Highball Coupon,ハイボールクーポン,高杯酒优惠券,高球雞尾酒優惠券
148,148,하이볼 한잔 무료 쿠폰,하이볼 한잔 무료 쿠폰,하이볼 한잔 무료 쿠폰,Free Cup of Highball Coupon,ハイボール1杯無料クーポン,高杯酒免费优惠券,高球雞尾酒單杯免費優惠券
149,149,하이볼잔 쿠폰,NaN,하이볼 잔 쿠폰,Highball Glass Coupon,ハイボール杯クーポン,高杯酒杯优惠券,高球雞尾酒杯優惠券
150,150,한우육회 쿠폰,한우육회 쿠폰,한우육회 쿠폰,Korean Beef Tartare Coupon,韓牛ユッケクーポン,韩牛生拌牛肉优惠券,韓牛生拌牛肉優惠券


In [384]:
result['num'] = re.search(r'\d+', str(result['title']))
result

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\2057934992.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['num'] = re.search(r'\d+', str(result['title']))


,promotion_id,coupon_id,restaurant_id,title,image,title_copy,EN,JP,CN,TW,numbers,num
0,323,426,1710029265,"1,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,1000,"<re.Match object; span=(0, 1), match='0'>"
2,335,438,1138224421,"2,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,2000,"<re.Match object; span=(0, 1), match='0'>"
7,394,497,1676849312,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_242/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,130g,"<re.Match object; span=(0, 1), match='0'>"
8,407,510,1991247186,삼겹살 100g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_209/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,100g,"<re.Match object; span=(0, 1), match='0'>"
9,414,517,1181287631,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_183/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,130g,"<re.Match object; span=(0, 1), match='0'>"
...,...,...,...,...,...,...,...,...,...,...,...,...
13595,201390,202225,1768782073,"1,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,1000,"<re.Match object; span=(0, 1), match='0'>"
13596,201393,202228,1131093932,"1,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,1000,"<re.Match object; span=(0, 1), match='0'>"
13597,201397,202232,1131093932,"2,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,2000,"<re.Match object; span=(0, 1), match='0'>"
13602,201513,202348,1254193590,10% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,10%,"<re.Match object; span=(0, 1), match='0'>"


In [415]:
# @ 에 대해서 숫자로 바꾸기 진행

mergedf = pd.merge(naver_coupon,refer[['title_copy','EN', 'JP', 'CN','TW']],on='title_copy',how='left')
result=mergedf[mergedf['EN'].notnull()]
import re
def extract_numbers_with_special_chars(text):
    return ''.join(re.findall(r'[\d+\-]+[a-zA-Z%]*', text))

# 각 행에 대해 함수 적용
result['numbers'] = result['title'].apply(extract_numbers_with_special_chars)
result['numbers'] = result['numbers'].replace('',None)
result.sample(20)
# 1. @ 먼저 바꾸기 2. title 숫자잇는거 3


result['new_column'] = result.apply(lambda x: True if pd.notna(x['numbers']) and '@' not in x['EN'] else False, axis=1)
result

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\3756782862.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['numbers'] = result['title'].apply(extract_numbers_with_special_chars)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\3756782862.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['numbers'] = result['numbers'].replace('',None)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\3756782862.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,promotion_id,coupon_id,restaurant_id,title,image,title_copy,EN,JP,CN,TW,numbers,new_column
0,323,426,1710029265,"1,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,1000,False
2,335,438,1138224421,"2,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,2000,False
7,394,497,1676849312,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_242/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,130g,True
8,407,510,1991247186,삼겹살 100g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_209/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,100g,True
9,414,517,1181287631,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_183/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,130g,True
...,...,...,...,...,...,...,...,...,...,...,...,...
13595,201390,202225,1768782073,"1,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,1000,False
13596,201393,202228,1131093932,"1,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,1000,False
13597,201397,202232,1131093932,"2,000원 할인 쿠폰",,원 할인 쿠폰,@ KRW discount coupon,@ウォン割引クーポン,@折扣优惠券,@韓元折扣優惠券,2000,False
13602,201513,202348,1254193590,10% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,10%,True


In [418]:
result[result['new_column'] == True].head(10)

,promotion_id,coupon_id,restaurant_id,title,image,title_copy,EN,JP,CN,TW,numbers,new_column
7,394,497,1676849312,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_242/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,130g,True
8,407,510,1991247186,삼겹살 100g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_209/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,100g,True
9,414,517,1181287631,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_183/166...,삼겹살 제공 쿠폰,Free Pork Belly Coupon,サムギョプサル提供クーポン,五花肉优惠券,提供五花肉優惠券,130g,True
16,651,756,35258262,5% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,5%,True
21,750,855,1604802194,10% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,10%,True
23,794,899,1992658571,50% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,50%,True
28,894,999,1784447310,5% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,5%,True
34,1061,1166,1661215357,10% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,10%,True
38,1212,1317,1398770169,10% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,10%,True
39,1216,1321,1999445482,5% 할인 쿠폰,,할인 쿠폰,Discount Coupon,割引クーポン,折扣优惠券,折扣優惠券,5%,True


In [422]:
result.loc[result['new_column'] == True, 'EN'] = result.apply(lambda x: f"{x['numbers']} {x['EN']}" if x['new_column'] else x['EN'], axis=1)
result.loc[result['new_column'] == True, 'JP'] = result.apply(lambda x: f"{x['numbers']} {x['JP']}" if x['new_column'] else x['JP'], axis=1)
result.loc[result['new_column'] == True, 'CN'] = result.apply(lambda x: f"{x['numbers']} {x['CN']}" if x['new_column'] else x['CN'], axis=1)
result.loc[result['new_column'] == True, 'TW'] = result.apply(lambda x: f"{x['numbers']} {x['TW']}" if x['new_column'] else x['TW'], axis=1)

In [423]:
result[result['new_column'] == True].head(10)

,promotion_id,coupon_id,restaurant_id,title,image,title_copy,EN,JP,CN,TW,numbers,new_column
7,394,497,1676849312,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_242/166...,삼겹살 제공 쿠폰,130g Free Pork Belly Coupon,130g サムギョプサル提供クーポン,130g 五花肉优惠券,130g 提供五花肉優惠券,130g,True
8,407,510,1991247186,삼겹살 100g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_209/166...,삼겹살 제공 쿠폰,100g Free Pork Belly Coupon,100g サムギョプサル提供クーポン,100g 五花肉优惠券,100g 提供五花肉優惠券,100g,True
9,414,517,1181287631,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_183/166...,삼겹살 제공 쿠폰,130g Free Pork Belly Coupon,130g サムギョプサル提供クーポン,130g 五花肉优惠券,130g 提供五花肉優惠券,130g,True
16,651,756,35258262,5% 할인 쿠폰,,할인 쿠폰,5% Discount Coupon,5% 割引クーポン,5% 折扣优惠券,5% 折扣優惠券,5%,True
21,750,855,1604802194,10% 할인 쿠폰,,할인 쿠폰,10% Discount Coupon,10% 割引クーポン,10% 折扣优惠券,10% 折扣優惠券,10%,True
23,794,899,1992658571,50% 할인 쿠폰,,할인 쿠폰,50% Discount Coupon,50% 割引クーポン,50% 折扣优惠券,50% 折扣優惠券,50%,True
28,894,999,1784447310,5% 할인 쿠폰,,할인 쿠폰,5% Discount Coupon,5% 割引クーポン,5% 折扣优惠券,5% 折扣優惠券,5%,True
34,1061,1166,1661215357,10% 할인 쿠폰,,할인 쿠폰,10% Discount Coupon,10% 割引クーポン,10% 折扣优惠券,10% 折扣優惠券,10%,True
38,1212,1317,1398770169,10% 할인 쿠폰,,할인 쿠폰,10% Discount Coupon,10% 割引クーポン,10% 折扣优惠券,10% 折扣優惠券,10%,True
39,1216,1321,1999445482,5% 할인 쿠폰,,할인 쿠폰,5% Discount Coupon,5% 割引クーポン,5% 折扣优惠券,5% 折扣優惠券,5%,True


In [424]:
result['TW'] = result['TW'].fillna('')

result['EN'] = result.apply(lambda x: x['EN'].replace('@', x['numbers']) if '@' in x['EN'] else x['EN'], axis=1)
result['JP'] = result.apply(lambda x: x['JP'].replace('@', x['numbers']) if '@' in x['JP'] else x['JP'], axis=1)
result['CN'] = result.apply(lambda x: x['CN'].replace('@', x['numbers']) if '@' in x['CN'] else x['CN'], axis=1)
result['TW'] = result.apply(lambda x: x['TW'].replace('@', x['numbers']) if '@' in x['TW'] else x['TW'], axis=1)
result

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\915538560.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['TW'] = result['TW'].fillna('')
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\915538560.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['EN'] = result.apply(lambda x: x['EN'].replace('@', x['numbers']) if '@' in x['EN'] else x['EN'], axis=1)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\915538560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,promotion_id,coupon_id,restaurant_id,title,image,title_copy,EN,JP,CN,TW,numbers,new_column
0,323,426,1710029265,"1,000원 할인 쿠폰",,원 할인 쿠폰,1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,1000韓元折扣優惠券,1000,False
2,335,438,1138224421,"2,000원 할인 쿠폰",,원 할인 쿠폰,2000 KRW discount coupon,2000ウォン割引クーポン,2000折扣优惠券,2000韓元折扣優惠券,2000,False
7,394,497,1676849312,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_242/166...,삼겹살 제공 쿠폰,130g Free Pork Belly Coupon,130g サムギョプサル提供クーポン,130g 五花肉优惠券,130g 提供五花肉優惠券,130g,True
8,407,510,1991247186,삼겹살 100g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_209/166...,삼겹살 제공 쿠폰,100g Free Pork Belly Coupon,100g サムギョプサル提供クーポン,100g 五花肉优惠券,100g 提供五花肉優惠券,100g,True
9,414,517,1181287631,삼겹살 130g 제공 쿠폰,https://ldb-phinf.pstatic.net/20220906_183/166...,삼겹살 제공 쿠폰,130g Free Pork Belly Coupon,130g サムギョプサル提供クーポン,130g 五花肉优惠券,130g 提供五花肉優惠券,130g,True
...,...,...,...,...,...,...,...,...,...,...,...,...
13595,201390,202225,1768782073,"1,000원 할인 쿠폰",,원 할인 쿠폰,1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,1000韓元折扣優惠券,1000,False
13596,201393,202228,1131093932,"1,000원 할인 쿠폰",,원 할인 쿠폰,1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,1000韓元折扣優惠券,1000,False
13597,201397,202232,1131093932,"2,000원 할인 쿠폰",,원 할인 쿠폰,2000 KRW discount coupon,2000ウォン割引クーポン,2000折扣优惠券,2000韓元折扣優惠券,2000,False
13602,201513,202348,1254193590,10% 할인 쿠폰,,할인 쿠폰,10% Discount Coupon,10% 割引クーポン,10% 折扣优惠券,10% 折扣優惠券,10%,True


In [446]:
# store_id 매
ids = ','.join(map(str, result['restaurant_id'].values))
ids

'1710029265,1138224421,1676849312,1991247186,1181287631,37572589,1309648231,35258262,1604802194,1777727682,1992658571,1373812378,1365169558,1784447310,17416024,1716583917,1661215357,1398770169,1999445482,1485660479,36096985,36096985,1631324110,1715293347,1723648430,1742989629,1516516804,1877483968,87369046,1991430161,1085351859,16318791,1793408538,1351681512,1959982694,35250719,992358775,1735239321,37215831,1119562188,13443386,1097389570,1452550151,1157068207,1656390575,1290337546,1289801812,1032839723,1899556844,1710326920,1316412520,1470170422,1228980982,1580296628,1060638038,1641742644,1117785215,1364892455,1115257229,1940698758,1538715663,1112763468,1053285768,1548655915,1287623895,1018283819,1726812693,1998104346,1132885434,1619683915,1142318742,38281375,1522702108,1700962438,1079582580,1995790487,1076529459,1730714864,1626579968,1752878881,1811779933,1731234633,1082963282,1387771082,1978599387,1191177839,1028388645,1832917671,1641792161,1075893501,1505529713,31992294,1084504901,3

In [453]:
with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:

        sql  = f'''SELECT R.RSTR_ID, R.SRC_ID , R.SRC, R2.RSTR_NM, R2.AREA_NM_1, R2.SRVIC_LANG_NM
        FROM RSTR_CONN R
        left JOIN RSTR R2 ON R.RSTR_ID = R2.ID
        WHERE R.SRC_ID in ({ids});'''
        
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())

result2 = pd.merge(result,df[['SRC_ID','RSTR_ID','RSTR_NM']],left_on='restaurant_id',right_on='SRC_ID',how='inner')
result2

# store_id 검색

ids2 = ','.join(map(str, result2['RSTR_ID'].values))
with dev_db_conn('redtable01') as conn:
    with conn.cursor() as curs:

        sql  = f'''SELECT R.RSTR_ID, R.SRC_ID , R.SRC, R2.RSTR_NM, R2.AREA_NM_1, R2.SRVIC_LANG_NM
        FROM RSTR_CONN R
        left JOIN RSTR R2 ON R.RSTR_ID = R2.ID
        WHERE R.RSTR_ID in ({ids2}) AND R.SRC ='live_db';'''
        
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
df

,RSTR_ID,SRC_ID,SRC,RSTR_NM,AREA_NM_1,SRVIC_LANG_NM
0,6042,363766,live_db,사이야,경기도,None
1,8108,203583,live_db,꼬꼬닭,부산광역시,None
2,10886,439231,live_db,명인만족,경상남도,None
3,12978,14531,live_db,섬진강민물장어,서울특별시,None
4,13106,35582,live_db,여의도황소곱창,서울특별시,None
...,...,...,...,...,...,...
1794,300136156,1071877,live_db,카페 국립공원,경기도,None
1795,300137003,1072714,live_db,토 닭,부산광역시,None
1796,300137199,1072907,live_db,파수타(pasuta),대구광역시,None
1797,300138856,1074548,live_db,회조아,경기도,None


In [459]:
final = pd.merge(result2,df, on='RSTR_ID',how='inner')
final.columns

Index(['promotion_id', 'coupon_id', 'restaurant_id', 'title', 'image',
       'title_copy', 'EN', 'JP', 'CN', 'TW', 'numbers', 'new_column',
       'SRC_ID_x', 'RSTR_ID', 'RSTR_NM_x', 'SRC_ID_y', 'SRC', 'RSTR_NM_y',
       'AREA_NM_1', 'SRVIC_LANG_NM'],
      dtype='object')

In [463]:
final2 = final[['title', 'EN', 'JP', 'CN','SRC_ID_y','AREA_NM_1']]
final2.drop_duplicates(subset=['SRC_ID_y','title','AREA_NM_1'], inplace=True)

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\3247607117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final2.drop_duplicates(subset=['SRC_ID_y','title','AREA_NM_1'], inplace=True)


In [975]:
final2

,title,EN,JP,CN,SRC_ID_y,AREA_NM_1
0,"1,000원 할인 쿠폰",1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,326368,경기도
1,삼겹살 130g 제공 쿠폰,130g Free Pork Belly Coupon,130g サムギョプサル提供クーポン,130g 五花肉优惠券,826120,서울특별시
2,삼겹살 100g 제공 쿠폰,100g Free Pork Belly Coupon,100g サムギョプサル提供クーポン,100g 五花肉优惠券,393454,경기도
3,삼겹살 130g 제공 쿠폰,130g Free Pork Belly Coupon,130g サムギョプサル提供クーポン,130g 五花肉优惠券,38236,서울특별시
4,"1,000원 할인 쿠폰",1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,311302,강원특별자치도
...,...,...,...,...,...,...
1910,"10,000원 할인 쿠폰",10000 KRW discount coupon,10000ウォン割引クーポン,10000折扣优惠券,69894,서울특별시
1911,음료 쿠폰,Drink Coupon,ドリンククーポン,饮料优惠券,900764,경기도
1912,"1,000원 할인 쿠폰",1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,542093,대구광역시
1913,"1,000원 할인 쿠폰",1000 KRW discount coupon,1000ウォン割引クーポン,1000折扣优惠券,768703,서울특별시


In [978]:
conn = dev_db_conn('dictionary')
curs = conn.cursor()
sql = """
SELECT SHIDO_NM,EN as "EN_shido",CN as "CN_shido" ,JP as "JP_shido" from shido;
"""
curs.execute(sql)
shido = pd.DataFrame(curs.fetchall())
shido

final2 = pd.merge(final2,shido, left_on='AREA_NM_1',right_on='SHIDO_NM',how='inner')

In [466]:
data_key = 'RT_RESTAURANT_COUPON_KO_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

RT_RESTAURANT_COUPON_KO_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_COUPON_KO_INFO의 최근 테이블은 RT_RESTAURANT_COUPON_KO_INFO_2022_12입니다.
테이블을 생성합니다.
테이블 생성을 완료하였습니다.
새 테이블에 insert 할 데이터를 확보합니다.


In [470]:
final2.columns

Index(['title', 'EN', 'JP', 'CN', 'SRC_ID_y', 'AREA_NM_1'], dtype='object')

#### 최종 데이터로 4개 테이블 업로드 진행

### RT_RESTAURANT_COUPON_KO_INFO

In [979]:
ko = final2[['AREA_NM_1','SRC_ID_y','title']]
EN = final2[['EN_shido','SRC_ID_y','EN']]
JP = final2[['JP_shido','SRC_ID_y','JP']]
CN = final2[['CN_shido','SRC_ID_y','CN']]

In [982]:
df_result_ko = ko.values.tolist()
df_result_EN = EN.values.tolist()
df_result_JP = JP.values.tolist()
df_result_CN = CN.values.tolist()

In [475]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

#final_result = df_result.values.tolist()

curs.executemany(sql,df_result_ko)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_COUPON_ZH_INFO

In [983]:
data_key = 'RT_RESTAURANT_COUPON_ZH_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_RESTAURANT_COUPON_ZH_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_COUPON_ZH_INFO의 최근 테이블은 RT_RESTAURANT_COUPON_ZH_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [984]:
table_nm

'RT_RESTAURANT_COUPON_ZH_INFO_202407'

In [985]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

#final_result = df_result.values.tolist()

curs.executemany(sql,df_result_CN)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_COUPON_JA_INFO 

In [988]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

#final_result = df_result.values.tolist()

curs.executemany(sql,df_result_JP)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_COUPON_EN_INFO


In [989]:
data_key = 'RT_RESTAURANT_COUPON_EN_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_RESTAURANT_COUPON_EN_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_COUPON_EN_INFO의 최근 테이블은 RT_RESTAURANT_COUPON_EN_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [992]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

#final_result = df_result.values.tolist()

curs.executemany(sql,df_result_EN)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_COUPON_JA_INFO 

In [ ]:
data_key = 'RT_RESTAURANT_COUPON_JA_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

RT_RESTAURANT_COUPON_JA_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_COUPON_JA_INFO의 최근 테이블은 RT_RESTAURANT_COUPON_JA_INFO_2022_09입니다.
테이블을 생성합니다.
테이블 생성을 완료하였습니다.
새 테이블에 insert 할 데이터를 확보합니다.


In [ ]:
conn = big_data_center()
curs = conn.cursor()

sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

#final_result = df_result.values.tolist()

curs.executemany(sql,df_result_JP)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')


insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### 메뉴 테이블 진행

In [ ]:
data_key = 'RT_RESTAURANT_KO_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

In [ ]:
# 참고 테이블 조회
data_key = 'RT_RESTAURANT_KO_INFO_202407'

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {}""".format(data_key)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())
summary

,CITY_NM,RSTRNT_ID,RESTAURANT_NM,RSTRNT_GIBUN_ADDR,RSTRNT_ROAD_ADDR,RSTRNT_TEL_NO,SUBWAY_NM,SUBWAY_LT,RSTRNT_LA,RSTRNT_LO
0,서울특별시,10012,명동정,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,02-3789-5130,을지로입구역,264.35,37.561597,126.988579
1,서울특별시,10013,명동 서서갈비,서울특별시 중구 명동2가 23,서울특별시 중구 명동10길 35-10,None,을지로입구역,185.92,37.562512,126.985993
2,서울특별시,10014,봉추찜닭,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,02-3789-9381,시청역,304.28,37.561065,126.982289
3,서울특별시,10016,명동한우방,서울특별시 중구 을지로2가 199-56,서울특별시 중구 명동9길 37-5,02-2267-8132,종각역,528.49,37.565390,126.983946
4,서울특별시,10017,설빙,서울특별시 서대문구 창천동 33-43,서울특별시 서대문구 연세로 23,02-325-8492,신촌역,351.59,37.557762,126.936690
...,...,...,...,...,...,...,...,...,...,...
867212,부산광역시,1191094,회사식당,부산광역시 강서구 지사동 1172,부산광역시 강서구 과학산단로 502,None,None,NaN,35.150177,128.816810
867213,대구광역시,1191095,후레쉬케터링 모토닉대구공장,대구광역시 달서구 신당동 1095-11,대구광역시 달서구 달서대로 530,053-382-3311,None,NaN,35.845846,128.493328
867214,부산광역시,1628785,차오란,부산광역시 해운대구 중동 1829,부산광역시 해운대구 달맞이길 30,0507-1356-1253,None,NaN,35.159937,129.169813
867215,부산광역시,1628786,야키토리 해공,부산광역시 수영구 민락동 24-6,부산광역시 수영구 민락본동로19번길 30-5,0507-1479-8334,None,NaN,35.158234,129.127114


In [ ]:
ids = ','.join(map(str , summary['RSTRNT_ID'].values.tolist()))
ids

In [571]:
with live_db_conn() as conn:
    with conn.cursor() as curs:
        start = time.time()
        sql =f"""
        SELECT COALESCE(s.address_road,s.address) as address , pts.lang, p.store_id , p.id, p.price, pts.`name`,
        GROUP_CONCAT(distinct pt.`name` ORDER BY pt.`name` SEPARATOR ',') AS tag_name , p4.`path`
        FROM product p
        INNER JOIN product_translate pts ON pts.product_id = p.id
        INNER JOIN store s ON p.store_id = s.id
        left JOIN product_tag p2 ON p.id = p2.product_id
        left JOIN product_tag_master p3 ON p2.product_tag_master_id = p3.id
        left JOIN product_tag_translate ptt ON ptt.product_tag_master_id = p3.id
        LEFT JOIN (SELECT product_tag_master_id, lang, name 
        			FROM product_tag_translate 
        			WHERE lang IN ('en', 'ja', 'zh-Hans', 'zh-Hant','ko')
        )pt ON pt.product_tag_master_id = p3.id AND pt.lang = pts.lang 
        INNER JOIN product_file p4 ON p4.product_id = p.id
        WHERE p.store_id in ({ids})
        AND p.`status` = 'normal' AND (s.sub_category = 1 or s.sub_category is null)
        GROUP BY 
        p.store_id, 
        p.id, 
        p.price, 
        pts.`name`;"""
        curs.execute(sql)
        df =  pd.DataFrame(curs.fetchall())
        print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)
df

쿼리 완료 
쿼리 작동 시간 : 115.16747069358826


,address,lang,store_id,id,price,name,tag_name,path
0,서울특별시 중구 삼일대로 299,en,10012,20024,65000.00,Gwibin Sang,None,/data/product/63190_pnBScus0MAd0zA1NgcgvT8woXB...
1,서울특별시 중구 삼일대로 299,ja,10012,20024,65000.00,クィビンサン,None,/data/product/60_FepERuWCqV3mETTQJ0x7DK3f1JgBp...
2,서울특별시 중구 삼일대로 299,zh-Hans,10012,20024,65000.00,贵宾席,None,/data/product/24915_crLDxFjdZTVgHXbpg5I30Z1v3z...
3,서울특별시 중구 삼일대로 299,ko,10012,20024,65000.00,귀빈상,None,/data/product/63190_pnBScus0MAd0zA1NgcgvT8woXB...
4,서울특별시 중구 삼일대로 299,en,10012,20025,43000.00,Myeongdong Jeongsik,Set Menu,/data/product/61_jyW6yIxtt0gTQO4N8wRhQlGxxzpeI...
...,...,...,...,...,...,...,...,...
118150,서울특별시 중구 명동6길 15,en,1631452,4454909,2000.00,[17:00~23:00/Dinner]Steamed Rice,"Rice,Steamed Rice",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...
118151,서울특별시 중구 명동6길 15,ja,1631452,4454909,2000.00,[17:00~23:00/ディナー]ご飯,"ご飯,米",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...
118152,서울특별시 중구 명동6길 15,zh-Hant,1631452,4454909,2000.00,[17:00~23:00/晚餐]米飯,"米,米飯",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...
118153,서울특별시 중구 명동6길 15,zh-Hans,1631452,4454909,2000.00,[17:00~23:00/晚餐]米饭,"大米,米饭",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...


In [576]:
df['address'] = df['address'].apply(lambda x : x.split(' ')[0] if pd.notnull(x) else x)

In [631]:
df2 = df[df['address'] != '']
df2 = df2[df2['address'] != '영등포구']
df2 = df2[df2['lang'] != 'zh-Hant']
df2 = df2[df2['price'] != 0]
df2 = df2[df2['price'] != 1]
#df['address_road'].value_counts()
df2['address'].replace('전남','전라남도',inplace=True)
df2['address'].replace('서울','서울특별시',inplace=True)
df2

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\410988367.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['address'].replace('전남','전라남도',inplace=True)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\410988367.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,address,lang,store_id,id,price,name,tag_name,path
0,서울특별시,en,10012,20024,65000.00,Gwibin Sang,None,/data/product/63190_pnBScus0MAd0zA1NgcgvT8woXB...
1,서울특별시,ja,10012,20024,65000.00,クィビンサン,None,/data/product/60_FepERuWCqV3mETTQJ0x7DK3f1JgBp...
2,서울특별시,zh-Hans,10012,20024,65000.00,贵宾席,None,/data/product/24915_crLDxFjdZTVgHXbpg5I30Z1v3z...
3,서울특별시,ko,10012,20024,65000.00,귀빈상,None,/data/product/63190_pnBScus0MAd0zA1NgcgvT8woXB...
4,서울특별시,en,10012,20025,43000.00,Myeongdong Jeongsik,Set Menu,/data/product/61_jyW6yIxtt0gTQO4N8wRhQlGxxzpeI...
...,...,...,...,...,...,...,...,...
118149,서울특별시,ko,1631452,4454908,3000.00,[17:00~23:00/저녁]1++한우 / 소고기 미역국 (S) [작은 사이즈],None,/data/product/71299_BsDrjanIc82PRoexFs2gs9lFq4...
118150,서울특별시,en,1631452,4454909,2000.00,[17:00~23:00/Dinner]Steamed Rice,"Rice,Steamed Rice",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...
118151,서울특별시,ja,1631452,4454909,2000.00,[17:00~23:00/ディナー]ご飯,"ご飯,米",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...
118153,서울특별시,zh-Hans,1631452,4454909,2000.00,[17:00~23:00/晚餐]米饭,"大米,米饭",/data/product/71300_mYQHzFCKeePwv4jNjcMThxZQVu...


In [ ]:
conn = dev_db_conn('dictionary')
curs = conn.cursor()

sql = """ 
        SELECT *
        from shido
        """
curs.execute(sql)
area1 = pd.DataFrame(curs.fetchall())


In [580]:
area1 = area1[['SHIDO_NM','EN','CN','JP']]
area1_melted = area1.melt(id_vars=['SHIDO_NM'], value_vars=['EN', 'CN', 'JP'], var_name='lang', value_name='translated_name')
area1_melted['lang'].replace({'EN':'en', 'CN':'zh-Hans', 'JP':'ja'}, inplace=True)

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\953464029.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  area1_melted['lang'].replace({'EN':'en', 'CN':'zh-Hans', 'JP':'ja'}, inplace=True)


In [609]:
result = df2.merge(area1_melted, how='left', left_on=['address', 'lang'], right_on=['SHIDO_NM', 'lang'])
result['translated_name'].fillna(result['address'],inplace=True)
result['path'] = result['path'].apply(lambda x : f'https://uihmdccdciav16245131.cdn.ntruss.com{x}?type=w&w=1024&quality=80&extopt=0')
result

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\381126795.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['translated_name'].fillna(result['address'],inplace=True)


,address,lang,store_id,id,price,name,tag_name,path,SHIDO_NM,translated_name
0,서울특별시,en,10012,20024,65000.00,Gwibin Sang,None,https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,Seoul
1,서울특별시,ja,10012,20024,65000.00,クィビンサン,None,https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,ソウル特別市
2,서울특별시,zh-Hans,10012,20024,65000.00,贵宾席,None,https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,首尔特别市
3,서울특별시,ko,10012,20024,65000.00,귀빈상,None,https://uihmdccdciav16245131.cdn.ntruss.com/da...,NaN,서울특별시
4,서울특별시,en,10012,20025,43000.00,Myeongdong Jeongsik,Set Menu,https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,Seoul
...,...,...,...,...,...,...,...,...,...,...
97976,서울특별시,ko,1631452,4454908,3000.00,[17:00~23:00/저녁]1++한우 / 소고기 미역국 (S) [작은 사이즈],None,https://uihmdccdciav16245131.cdn.ntruss.com/da...,NaN,서울특별시
97977,서울특별시,en,1631452,4454909,2000.00,[17:00~23:00/Dinner]Steamed Rice,"Rice,Steamed Rice",https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,Seoul
97978,서울특별시,ja,1631452,4454909,2000.00,[17:00~23:00/ディナー]ご飯,"ご飯,米",https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,ソウル特別市
97979,서울특별시,zh-Hans,1631452,4454909,2000.00,[17:00~23:00/晚餐]米饭,"大米,米饭",https://uihmdccdciav16245131.cdn.ntruss.com/da...,서울특별시,首尔特别市


In [610]:
result = result.fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])

In [611]:
final = result[['translated_name','store_id','id','price','name','tag_name','path','lang']]

#### 최종 데이터로 4개 테이블 업로드 진행

### RT_RESTAURANT_MENU_KO_INFO

In [625]:
data_key = 'RT_RESTAURANT_MENU_KO_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary

table_nm = '{}_{}'.format(data_key,date_string)

'''print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')'''

RT_RESTAURANT_MENU_KO_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_MENU_KO_INFO의 최근 테이블은 RT_RESTAURANT_MENU_KO_INFO_202407입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [627]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_RESTAURANT_MENU_KO_INFO_2022_12")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary

table_nm = '{}_{}'.format(data_key,date_string)

In [629]:
table_nm

'RT_RESTAURANT_MENU_KO_INFO_202407'

In [628]:
col_nm

'CITY_NM,RSTRNT_ID,MENU_ID,MENU_PC,MENU_NM,MENU_TAG,MENU_IMAGE'

In [630]:
ko = final[final['lang'] == 'ko'].drop(columns='lang')
final_result = ko.values.tolist()

conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_MENU_JA_INFO


In [563]:
data_key = 'RT_RESTAURANT_MENU_JA_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

RT_RESTAURANT_MENU_JA_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_MENU_JA_INFO의 최근 테이블은 RT_RESTAURANT_MENU_JA_INFO_2022_12입니다.


'print(\'테이블을 생성합니다.\')\nsql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;\n;""".format(table_nm,orig_nm)\n\ncurs.execute(sql)\nconn.commit()\nprint(\'테이블 생성을 완료하였습니다.\')\nprint(\'새 테이블에 insert 할 데이터를 확보합니다.\')'

In [639]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_RESTAURANT_MENU_JA_INFO_2022_12")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_RESTAURANT_MENU_JA_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(table_nm)

In [642]:
ja = final[final['lang'] == 'ja'].drop(columns='lang')
final_result = ja.values.tolist()

conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


In [564]:
table_nm

'RT_RESTAURANT_MENU_JA_INFO_202407'

### RT_RESTAURANT_MENU_ZH_INFO

In [559]:
data_key = 'RT_RESTAURANT_MENU_ZH_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

RT_RESTAURANT_MENU_ZH_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_MENU_ZH_INFO의 최근 테이블은 RT_RESTAURANT_MENU_ZH_INFO_2022_12입니다.
테이블을 생성합니다.
테이블 생성을 완료하였습니다.
새 테이블에 insert 할 데이터를 확보합니다.


In [647]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_RESTAURANT_MENU_ZH_INFO_2022_12")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_RESTAURANT_MENU_ZH_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(table_nm)

CITY_NM,RSTRNT_ID,MENU_ID,MENU_PC,MENU_NM,MENU_TAG,MENU_IMAGE
RT_RESTAURANT_MENU_ZH_INFO_202407


In [648]:
zhHans = final[final['lang'] == 'zh-Hans'].drop(columns='lang')
final_result = zhHans.values.tolist()

conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_RESTAURANT_MENU_EN_INFO


In [561]:
data_key = 'RT_RESTAURANT_MENU_EN_INFO'
today = datetime.now()
date_string = today.strftime("%Y%m")

# 최근 table 확인
print('{}의 최근 테이블을 확인합니다.'.format(data_key))
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM BIGDATA_SUMMARY"""

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

orig_nm = summary[summary['TABLE_NM']==data_key]['RECENT_TABLE_NM'].iloc[0]
print('확인완료')
print('{}의 최근 테이블은 {}입니다.'.format(data_key,orig_nm))

conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format(orig_nm)

curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

del summary

table_nm = '{}_{}'.format(data_key,date_string)

print('테이블을 생성합니다.')
sql = """CREATE TABLE IF NOT EXISTS `{}` LIKE `{}`;
;""".format(table_nm,orig_nm)

curs.execute(sql)
conn.commit()
print('테이블 생성을 완료하였습니다.')
print('새 테이블에 insert 할 데이터를 확보합니다.')

RT_RESTAURANT_MENU_EN_INFO의 최근 테이블을 확인합니다.
확인완료
RT_RESTAURANT_MENU_EN_INFO의 최근 테이블은 RT_RESTAURANT_MENU_EN_INFO_2022_12입니다.
테이블을 생성합니다.
테이블 생성을 완료하였습니다.
새 테이블에 insert 할 데이터를 확보합니다.


In [649]:
conn = big_data_center()
curs = conn.cursor()

sql = """SELECT * FROM {} limit 2""".format("RT_RESTAURANT_MENU_EN_INFO_2022_12")
curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

col_nm = ','.join(summary.columns.tolist())
col_cnt = ','.join(['%s' for c in range(len(summary.columns))])

#del summary
data_key = 'RT_RESTAURANT_MENU_EN_INFO'
table_nm = '{}_{}'.format(data_key,date_string)
print(col_nm)
print(table_nm)

CITY_NM,RSTRNT_ID,MENU_ID,MENU_PC,MENU_NM,MENU_TAG,MENU_IMAGE
RT_RESTAURANT_MENU_EN_INFO_202407


In [651]:
en = final[final['lang'] == 'en'].drop(columns='lang')
final_result = en.values.tolist()

conn = big_data_center()
curs = conn.cursor()
sql = """
INSERT INTO {} ({}) VALUES ({})
""".format(table_nm,col_nm,col_cnt)

curs.executemany(sql,final_result)
conn.commit()

curs.close()
conn.close()
print('insert를 완료하였습니다.')
print('빅데이터 갱신관리 테이블 데이터를 업데이트합니다.')
conn = big_data_center()
curs = conn.cursor()
sql = """
UPDATE BIGDATA_SUMMARY SET RECENT_TABLE_NM=%s,UPDATED_AT=%s WHERE TABLE_NM=%s
"""
curs.execute(sql,(table_nm,today,data_key))
conn.commit()
curs.close()
conn.close()
print('데이터 생성 및 업데이트를 완료하였습니다.')

insert를 완료하였습니다.
빅데이터 갱신관리 테이블 데이터를 업데이트합니다.
데이터 생성 및 업데이트를 완료하였습니다.


### RT_HOTEL_NEARBY_RSTRNT_INFO

In [ ]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_HOTEL_NEARBY_RSTRNT_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

## 테이블 정의서 작성

In [ ]:
conn = big_data_center()
curs = conn.cursor()
tables = [
    "RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_202406",
    "RT_INFN_ACP_POSBL_RSTRNT_INFO_202407",
    "RT_VGTR_RSTRNT_INFO_202407",
    "RT_FRNR_RESVE_POSBL_RSTRNT_INFO_202407",
    "RT_PET_ACP_POSBL_RSTRNT_INFO_202407",
    "RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO_202407",
    "RT_LCLT_ARPRT_RSTRNT_POPULAR_MENU_INFO_202407",
    "RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_202407",
    "RT_LDGBSSH_CFR_GBRSTRNT_POPULAR_MENU_INFO_202407",
    "RT_TURSM_LMK_CFR_RSTRNT_INFO_202407",
    "RT_MUSLIM_RSTRNT_INFO_202407",
    "RT_HALAL_MENU_INFO_202407",
    "RT_FRNR_TURIST_BEST_RSTRNT_INFO_202407",
    "RT_HOTEL_NEARBY_RSTRNT_INFO_202407",
    "RT_LOCAL_ARPRT_RSTRNT_INFO_202407",
    "RT_LOCGOV_CRTFC_RSTRNT_INFO_202407",
    "RT_GLOBAL_OTA_FDTURSM_INFO_202407",
    "RT_CN_MENU_INFO_202407",
    "RT_JP_MENU_INFO_202407",
    "RT_PEK_RSTRNT_INFO_202407",
    "RT_SHA_RSTRNT_INFO_202407",
    "RT_TYO_RSTRNT_INFO_202407",
    "RT_KFD_RSTRNT_INFO_202407",
    "RT_KFD_RSTRNT_SVC_INFO_202407",
    "RT_KFD_MENU_INFO_202407",
    "RT_KFD_MENU_ATTR_INFO_202407",
    "RT_KFD_TRDIT_RSTRNT_INFO_202407",
    "RT_RESTAURANT_KO_INFO_202407",
    "RT_RESTAURANT_ZH_INFO_202407",
    "RT_RESTAURANT_JA_INFO_202407",
    "RT_RESTAURANT_EN_INFO_202407",
    "RT_GLOBAL_RESTAURANT_INFO_202407",
    "RT_RESTAURANT_MENU_KO_INFO_202407",
    "RT_RESTAURANT_MENU_ZH_INFO_202407",
    "RT_RESTAURANT_MENU_JA_INFO_202407",
    "RT_RESTAURANT_MENU_EN_INFO_202407",
    "RT_RESTAURANT_SALE_INFO_202407",
    "RT_RESTAURANT_COUPON_KO_INFO_202407",
    "RT_RESTAURANT_COUPON_ZH_INFO_202407",
    "RT_RESTAURANT_COUPON_JA_INFO_202407",
    "RT_RESTAURANT_COUPON_EN_INFO_202407",
    "RT_KOREAN_LIQUOR_INFO_202407",
    "RT_WINE_INFO_202407",
    "RT_SAKE_INFO_202407",
    "RT_RESTAURANT_COMMERCIAL_AREA_INFO_202407"
]

table_counts = {}
for i in tables:

    sql = """
        SELECT 
        c.TABLE_NAME,
        c.COLUMN_NAME,
        CONCAT(c.DATA_TYPE, IFNULL(CONCAT('(', c.CHARACTER_MAXIMUM_LENGTH, ')'), '')) AS DATA_TYPE_LENGTH,
        c.IS_NULLABLE,
        c.COLUMN_COMMENT,
        kcu.CONSTRAINT_NAME AS KEY_TYPE,
        c.COLUMN_DEFAULT
    FROM 
        INFORMATION_SCHEMA.COLUMNS c
    LEFT JOIN 
        INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu ON c.TABLE_NAME = kcu.TABLE_NAME 
            AND c.COLUMN_NAME = kcu.COLUMN_NAME 
            AND c.TABLE_SCHEMA = kcu.TABLE_SCHEMA
    WHERE 
        c.TABLE_SCHEMA = 'redtable_bigdata_center'
        AND c.TABLE_NAME = 'RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_202407'
    ORDER BY
        c.TABLE_NAME, c.COLUMN_NAME;"""
    curs.execute(f"SELECT {i} FROM information_schema.tables WHERE table_schema = 'redtable_bigdata_center';")
    table = curs.fetchall()
    tables.append(table)
    curs.execute(sql)
summary = pd.DataFrame(curs.fetchall())

#summary
tables

In [1121]:
conn = big_data_center()
curs = conn.cursor()
tables = [
    "RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_202406",
    "RT_INFN_ACP_POSBL_RSTRNT_INFO_202407",
    "RT_VGTR_RSTRNT_INFO_202407",
    "RT_FRNR_RESVE_POSBL_RSTRNT_INFO_202407",
    "RT_PET_ACP_POSBL_RSTRNT_INFO_202407",
    "RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO_202407",
    "RT_LCLT_ARPRT_RSTRNT_POPULAR_MENU_INFO_202407",
    "RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_202407",
    "RT_LDGBSSH_CFR_GBRSTRNT_POPULAR_MENU_INFO_202407",
    "RT_TURSM_LMK_CFR_RSTRNT_INFO_202407",
    "RT_MUSLIM_RSTRNT_INFO_202407",
    "RT_HALAL_MENU_INFO_202407",
    "RT_FRNR_TURIST_BEST_RSTRNT_INFO_202407",
    "RT_HOTEL_NEARBY_RSTRNT_INFO_202407",
    "RT_LOCAL_ARPRT_RSTRNT_INFO_202407",
    "RT_LOCGOV_CRTFC_RSTRNT_INFO_202407",
    "RT_GLOBAL_OTA_FDTURSM_INFO_202407",
    "RT_CN_MENU_INFO_202407",
    "RT_JP_MENU_INFO_202407",
    "RT_PEK_RSTRNT_INFO_202407",
    "RT_SHA_RSTRNT_INFO_202407",
    "RT_TYO_RSTRNT_INFO_202407",
    "RT_KFD_RSTRNT_INFO_202407",
    "RT_KFD_RSTRNT_SVC_INFO_202407",
    "RT_KFD_MENU_INFO_202407",
    "RT_KFD_MENU_ATTR_INFO_202407",
    "RT_KFD_TRDIT_RSTRNT_INFO_202407",
    "RT_RESTAURANT_KO_INFO_202407",
    "RT_RESTAURANT_ZH_INFO_202407",
    "RT_RESTAURANT_JA_INFO_202407",
    "RT_RESTAURANT_EN_INFO_202407",
    "RT_GLOBAL_RESTAURANT_INFO_202407",
    "RT_RESTAURANT_MENU_KO_INFO_202407",
    "RT_RESTAURANT_MENU_ZH_INFO_202407",
    "RT_RESTAURANT_MENU_JA_INFO_202407",
    "RT_RESTAURANT_MENU_EN_INFO_202407",
    "RT_RESTAURANT_SALE_INFO_202407",
    "RT_RESTAURANT_COUPON_KO_INFO_202407",
    "RT_RESTAURANT_COUPON_ZH_INFO_202407",
    "RT_RESTAURANT_COUPON_JA_INFO_202407",
    "RT_RESTAURANT_COUPON_EN_INFO_202407",
    "RT_KOREAN_LIQUOR_INFO_202407",
    "RT_WINE_INFO_202407",
    "RT_SAKE_INFO_202407",
    "RT_RESTAURANT_COMMERCIAL_AREA_INFO_202407"
]

# 테이블별 행 개수를 저장할 딕셔너리
table_counts = []

# 각 테이블의 행 개수를 구하는 반복문
for table in tables:
    try:
        curs.execute(f"SELECT COUNT(*) FROM {table}")
        result = curs.fetchall()
        if result:
            count = result[0]
        else:
            count = 0
        table_counts.append((table, count))
    except :
        pass

# 결과 출력
table_counts
# 연결 종료
curs.close()
conn.close()

In [1123]:
df_counts = pd.DataFrame(table_counts, columns=['table_name', 'row_count'])
df_counts.to_excel("데이터수.xlsx")

In [1104]:
summary.to_excel("테이블정의서3.xlsx")

### 컬럼 정의서 작성

In [1137]:
colum = pd.read_excel("데이터품질인증 사전준비사항_20240625.xlsx", sheet_name='02.컬럼 정의서_2024')
colum['테이블 (영문명)'].fillna(method='ffill', inplace=True)

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\1556540531.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  colum['테이블 (영문명)'].fillna(method='ffill', inplace=True)
C:\Users\레드테이블\AppData\Local\Temp\ipykernel_624\1556540531.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  colum['테이블 (영문명)'].fillna(method='ffill', inplace=True)


In [1139]:
refer = pd.read_excel("데이터품질인증 사전준비사항_20240625.xlsx", sheet_name='참고')
refer

,데이터명,테이블명
0,연도별 식당 업종 및 업태 분석 정보,RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INF...
1,유아 동반 가능 식당 정보,RT_INFN_ACP_POSBL_RSTRNT_INFO_202407
2,채식 식당 정보(베지테리언),RT_VGTR_RSTRNT_INFO_202407
3,외국인 예약 가능 식당 정보,RT_FRNR_RESVE_POSBL_RSTRNT_INFO_202407
4,반려견 동반 식당 정보,RT_PET_ACP_POSBL_RSTRNT_INFO_202407
5,온라인(비대면)주문서비스 제공 카페 베이커리 정보,RT_ONLINE_ORDER_SVC_PROVD_CAFE_BAKERY_INFO_202407
6,지방 공항 식당 인기 메뉴 데이터,RT_LCLT_ARPRT_RSTRNT_POPULAR_MENU_INFO_202407
7,지자체 인증 식당 메뉴 정보,RT_LOCGOV_CRTFC_RSTRNT_MENU_INFO_202407
8,숙박업소 주변 맛집 인기 메뉴 정보,RT_LDGBSSH_CFR_GBRSTRNT_POPULAR_MENU_INFO_202407
9,관광 랜드마크 주변 식당 정보,RT_TURSM_LMK_CFR_RSTRNT_INFO_202407


In [1141]:
pd.merge(colum,refer,left_on='테이블 (영문명)', right_on='테이블명',how='left').to_excel("최종.xlsx")

### ERD 작성을 위한 CREATE 코드 출력

https://dbdiagram.io/d


In [1184]:
conn = big_data_center()
curs = conn.cursor()

create_statements = []    
for table in tables:
        sql = f"SHOW CREATE TABLE {table};"
        curs.execute(sql)
        result = curs.fetchone()
        print(result)
        if result:
            create_statements.append(result)
            #rint(summary)
    
curs.close()
conn.close()


A = pd.DataFrame(create_statements)
A.to_excel("ERD_create_code.xlsx")


{'Table': 'RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_202406', 'Create Table': "CREATE TABLE `RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_202406` (\n  `ID` varchar(50) collate utf8_unicode_ci NOT NULL COMMENT 'RSTR ID',\n  `AREA_FLAG_NM` varchar(200) collate utf8_unicode_ci default NULL COMMENT '광역시도',\n  `OPBIZ_YEAR` varchar(4) collate utf8_unicode_ci default NULL COMMENT '사업자등록년도',\n  `KORFOOD_RSTRNT_CO` decimal(38,0) default NULL COMMENT 'RSTR LABEL MASTER 의 한식',\n  `WESFOOD_RSTRNT_CO` decimal(38,0) default NULL COMMENT 'RSTR LABEL MASTER 의 양식',\n  `CHNFOOD_RSTRNT_CO` decimal(38,0) default NULL COMMENT 'RSTR LABEL MASTER 의 중식',\n  `JPNFOOD_RSTRNT_CO` decimal(38,0) default NULL COMMENT 'RSTR LABEL MASTER 의 일식',\n  `CAFE_CO` decimal(38,0) default NULL COMMENT 'RSTR LABEL MASTER 의 카페',\n  `RSTAREA_CO` decimal(38,0) default NULL COMMENT 'RSTR_NM 의 ''휴게소'' 수',\n  `PUB_CO` decimal(38,0) default NULL COMMENT 'BSNS_STATM_BZCND_NM의 ''정종/대포집/소주방'', ''호프/통닭''',\n  `ETC_RSTRNT_CO` deci

In [1171]:
pd.DataFrame(create_statements)

ValueError: Must pass 2-d input. shape=(45, 1, 2)

In [1159]:
create

[   Table  Create Table
 0  Table  Create Table,
    Table  Create Table
 0  Table  Create Table,
    Table  Create Table
 0  Table  Create Table,
    Table  Create Table
 0  Table  Create Table]

In [1156]:
pd.DataFrame(create)

ValueError: Must pass 2-d input. shape=(4, 1, 2)